# Script

## Bibliotecas

In [18866]:
import pandas as pd
import mysql.connector
import numpy as np
import importlib
import clubes
import os

In [18867]:
import clubes
print(clubes.__file__)


c:\Pessoal\Data_science\Projetos\sofascore\analises\clubes.py


Recarregando a informação que consta no arquivo "clubes.py".

In [18868]:
importlib.reload(clubes)
print(clubes.casa)

Al-Ettifaq


## Partidas

### Configurando o banco de dados e carregando a consulta

In [18869]:
config = {
    "host": "127.0.0.1",
    "user": "admin",
    "password": "1234",
    "database": "bet_dados"
}

In [18870]:
conn = mysql.connector.connect(**config)

In [18871]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols
FROM
	partidas
WHERE
	(casa = "{clubes.casa}" OR fora = "{clubes.casa}" OR casa = "{clubes.fora}" OR fora = "{clubes.fora}");
"""

In [18872]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [18873]:
conn.close()

In [18874]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2024-08-22,2024/2025,12545108,Saudita,Al-Nassr,1,1,Al-Raed,1.14,8.00,11.00,Empate,2
1,2024-08-24,2024/2025,12544932,Saudita,Al-Shabab,0,1,Al-Ettifaq,2.27,3.40,2.95,Al-Ettifaq,1
2,2024-08-27,2024/2025,12618489,Saudita,Al-Fayha,1,4,Al-Nassr,12.00,9.50,1.11,Al-Nassr,5
3,2024-08-28,2024/2025,12618497,Saudita,Al-Ettifaq,1,0,Al-Okhdood,1.38,4.33,7.50,Al-Ettifaq,1
4,2024-09-13,2024/2025,12625053,Saudita,Al-Nassr,1,1,Al-Ahli,1.70,3.90,3.90,Empate,2


### Tratando as informações

### Casa Atual

Criando uma cópia do DataFrame

In [18875]:
mask = (df["casa"] == clubes.casa) | (df["fora"] == clubes.casa)
casa = df.loc[mask].copy()

casa

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
1,2024-08-24,2024/2025,12544932,Saudita,Al-Shabab,0,1,Al-Ettifaq,2.27,3.40,2.95,Al-Ettifaq,1
3,2024-08-28,2024/2025,12618497,Saudita,Al-Ettifaq,1,0,Al-Okhdood,1.38,4.33,7.50,Al-Ettifaq,1
5,2024-09-14,2024/2025,12625048,Saudita,Al-Fateh,1,2,Al-Ettifaq,2.80,3.25,2.35,Al-Ettifaq,3
6,2024-09-20,2024/2025,12625039,Saudita,Al-Ettifaq,0,3,Al-Nassr,4.50,4.00,1.60,Al-Nassr,3
8,2024-09-29,2024/2025,12825259,Saudita,Al-Taawoun,1,1,Al-Ettifaq,2.35,3.10,2.88,Empate,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,2025-10-24,2025,14429574,Saudita,Al-Fateh,2,1,Al-Ettifaq,2.40,3.60,2.75,Al-Fateh,3
388,2025-10-31,2025,14771807,Saudita,Al-Ettifaq,2,2,Al-Hazem,2.05,3.50,3.40,Empate,4
390,2025-11-08,2025,14771811,Saudita,Al-Shabab,1,1,Al-Ettifaq,1.95,3.50,3.40,Empate,2
392,2025-11-22,2025,14771897,Saudita,Al-Ettifaq,3,2,Al-Fayha,2.80,3.30,2.38,Al-Ettifaq,5


In [18876]:
casa.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
1,2024-08-24,2024/2025,12544932,Saudita,Al-Shabab,0,1,Al-Ettifaq,2.27,3.40,2.95,Al-Ettifaq,1
3,2024-08-28,2024/2025,12618497,Saudita,Al-Ettifaq,1,0,Al-Okhdood,1.38,4.33,7.50,Al-Ettifaq,1
5,2024-09-14,2024/2025,12625048,Saudita,Al-Fateh,1,2,Al-Ettifaq,2.80,3.25,2.35,Al-Ettifaq,3
6,2024-09-20,2024/2025,12625039,Saudita,Al-Ettifaq,0,3,Al-Nassr,4.50,4.00,1.60,Al-Nassr,3
8,2024-09-29,2024/2025,12825259,Saudita,Al-Taawoun,1,1,Al-Ettifaq,2.35,3.10,2.88,Empate,2


Selecionando a temporada atual

In [18877]:
casa = casa[casa["temporada"] == "2025"]

Contando os jogos

In [18878]:
jogos_casa = casa.shape[0]

jogos_casa

11

Contando as vitórias

In [18879]:
vitorias = casa[casa["vencedor"] == clubes.casa]

vitorias_casa = vitorias.shape[0]

vitorias_casa

4

Contando os empates

In [18880]:
empates = casa[casa["vencedor"] == "Empate"]

empates_casa = empates.shape[0]

empates_casa

4

Contando as derrotas

In [18881]:
derrotas = casa[(casa["vencedor"] != clubes.casa) & (casa["vencedor"] != "Empate")]

derrotas_casa = derrotas.shape[0]

derrotas_casa

3

Gols marcados

In [18882]:
gols_casa = casa.loc[casa["casa"] == clubes.casa, "gols_casa"].sum()
gols_fora = casa.loc[casa["fora"] == clubes.casa, "gols_fora"].sum()

gols_marcados_casa = gols_casa + gols_fora

gols_marcados_casa

17

Gols sofridos

In [18883]:
gols_sofridos_casa = casa.loc[casa["casa"] == clubes.casa, "gols_fora"].sum()
gols_sofridos_fora = casa.loc[casa["fora"] == clubes.casa, "gols_casa"].sum()

gols_sofridos_casa = gols_sofridos_casa + gols_sofridos_fora

gols_sofridos_casa

20

Aproveitamento

In [18884]:
if jogos_casa == 0:
    aproveitamento_casa = 0
else:
    aproveitamento_casa = round((vitorias_casa * 3 + empates_casa) / (jogos_casa * 3) * 100, 2)

aproveitamento_casa

48.48

Over 2.5 gols

In [18885]:
over_2_casa = casa[casa["total_gols"] >= 3].shape[0]

over_2_casa

8

% de jogos com 3 ou mais gols

In [18886]:
if jogos_casa == 0:
    prob_over_2_casa = 0
else:
    prob_over_2_casa = round((over_2_casa / jogos_casa) * 100, 2)

prob_over_2_casa

72.73

Under 2.5 gols

In [18887]:
under_2_casa = casa[casa["total_gols"] < 3].shape[0]

under_2_casa

3

% de jogos com menos de 3 gols

In [18888]:
if jogos_casa == 0:
    prob_under_2_casa = 0
else:
    prob_under_2_casa = round((under_2_casa / jogos_casa) * 100, 2)

prob_under_2_casa

27.27

Odd média

In [18889]:
odd_media_casa = casa[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_fora = casa[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_casa = round((odd_media_casa + odd_media_fora) / 2, 2)

odd_media_casa

3.01

Odd média quando venceu

In [18890]:
odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_vitoria_casa = round(odd_media_vitoria_casa + odd_media_vitoria_fora / 2, 2)

odd_media_vitoria_casa

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\524038383.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\524038383.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()


3.4

Contando as goleadas

In [18891]:
goleadas_mandante_casa = casa[(casa["casa"] == clubes.casa) & (casa["gols_casa"] >= 4)].shape[0]
goleadas_mandante_fora = casa[(casa["fora"] == clubes.casa) & (casa["gols_fora"] >= 4)].shape[0]

goleadas_casa = goleadas_mandante_casa + goleadas_mandante_fora

goleadas_casa

0

% de goleadas aplicadas

In [18892]:
if jogos_casa == 0:
    prob_goleada_casa = 0
else:
    prob_goleada_casa = round((goleadas_casa / jogos_casa) * 100, 2)

prob_goleada_casa

0.0

Montando o DataFrame do time da casa

In [18893]:
dados = {
    "Jogos": jogos_casa,
    "Vitórias": vitorias_casa,
    "Empates": empates_casa,
    "Derrotas": derrotas_casa,
    "Gols marcados": gols_marcados_casa,
    "Gols sofridos": gols_sofridos_casa,
    "Aproveitamento": aproveitamento_casa,
    "Over 2.5 gols": over_2_casa,
    "Probabilidade de over 2.5 gols": prob_over_2_casa,
    "Under 2.5 gols": under_2_casa,
    "Probabilidade de under 2.5 gols": prob_under_2_casa,
    "Odd média": odd_media_casa,
    "Odd média quando venceu": odd_media_vitoria_casa,
    "Goleadas": goleadas_casa,
    "Probabilidade de golear": prob_goleada_casa,
    "Goleadas como mandante": goleadas_mandante_casa,
    "Goleadas como visitante": goleadas_mandante_fora
}

In [18894]:
ordem = [
    "Jogos", "Vitórias", "Empates", "Derrotas",
    "Gols marcados", "Gols sofridos",
    "Aproveitamento",
    "Over 2.5 gols", "Probabilidade de over 2.5 gols",
    "Under 2.5 gols", "Probabilidade de under 2.5 gols",
    "Odd média", "Odd média quando venceu",
    "Goleadas", "Probabilidade de golear",
]

faltando = [k for k in ordem if k not in dados]
if faltando:
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")


dados_ordenados = {k: dados[k] for k in ordem}
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=[clubes.casa]
)


variaveis_float = {
    "aproveitamento",
    "probabilidade de over 2.5 gols",
    "probabilidade de under 2.5 gols",
    "odd média",
    "odd média quando venceu",
    "probabilidade de golear",
}

def _fmt(nome, valor):
    if str(nome).lower() in variaveis_float:
        return f"{float(valor):.2f}"
    else:
        return f"{int(valor)}"

df_final_partidas[clubes.casa] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[clubes.casa])
]

# Índice vira coluna "Informações"
tabela_casa = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [18895]:
tabela_casa.style.hide(axis="index")

Informações,Al-Ettifaq
Jogos,11
Vitórias,4
Empates,4
Derrotas,3
Gols marcados,17
Gols sofridos,20
Aproveitamento,48.48
Over 2.5 gols,8
Probabilidade de over 2.5 gols,72.73
Under 2.5 gols,3


### Fora Atual

In [18896]:
fora = df[(df["casa"] == clubes.fora) | (df["fora"] == clubes.fora)]

Selecionando a temporada atual

In [18897]:
fora = fora[fora["temporada"] == "2025"]

Contando os jogos

In [18898]:
jogos_fora = fora.shape[0]

jogos_fora

14

Contando as vitórias

In [18899]:
vitorias = fora[fora["vencedor"] == clubes.fora]

vitorias_fora = vitorias.shape[0]

vitorias_fora

12

Contando os empates

In [18900]:
empates = fora[fora["vencedor"] == "Empate"]

empates_fora = empates.shape[0]

empates_fora

1

Contando as derrotas

In [18901]:
derrotas = fora[(fora["vencedor"] != clubes.fora) & (fora["vencedor"] != "Empate")]

derrotas_fora = derrotas.shape[0]

derrotas_fora

1

Gols marcados

In [18902]:
gols_casa = fora.loc[fora["casa"] == clubes.fora, "gols_casa"].sum()
gols_fora = fora.loc[fora["fora"] == clubes.fora, "gols_fora"].sum()

gols_marcados_fora = gols_casa + gols_fora

gols_marcados_fora

42

Gols sofridos

In [18903]:
gols_sofridos_casa = fora.loc[fora["casa"] == clubes.fora, "gols_fora"].sum()
gols_sofridos_fora = fora.loc[fora["fora"] == clubes.fora, "gols_casa"].sum()

gols_sofridos_fora = gols_sofridos_casa + gols_sofridos_fora

gols_sofridos_fora

10

Aproveitamento

In [18904]:
if jogos_fora == 0:
    aproveitamento_fora = 0
else:
    aproveitamento_fora = round((vitorias_fora * 3 + empates_fora) / (jogos_fora * 3) * 100, 2)

aproveitamento_fora

88.1

Over 2.5 gols

In [18905]:
over_2_fora = fora[fora["total_gols"] >= 3].shape[0]

over_2_fora

11

% de jogos com 3 ou mais gols

In [18906]:
if jogos_fora == 0:
    prob_over_2_fora = 0
else:
    prob_over_2_fora = round((over_2_fora / jogos_fora) * 100, 2)

prob_over_2_fora

78.57

Under 2.5 gols

In [18907]:
under_2_fora = fora[fora["total_gols"] < 3].shape[0]

under_2_fora

3

% de jogos com menos de 3 gols

In [18908]:
if jogos_fora == 0:
    prob_under_2_fora = 0
else:
    prob_under_2_fora = round((under_2_fora / jogos_fora) * 100, 2)

prob_under_2_fora

21.43

Odd média

In [18909]:
odd_media_casa = fora[fora["casa"] == clubes.fora]["odd_casa"].mean()
odd_media_fora = fora[fora["fora"] == clubes.fora]["odd_fora"].mean()

odd_media_fora = round((odd_media_casa + odd_media_fora) / 2, 2)

odd_media_fora

1.35

Odd média quando venceu

In [18910]:
odd_media_vitoria_casa = fora[(fora["casa"] == clubes.fora) & (fora["vencedor"] == clubes.fora)]["odd_casa"].mean()
odd_media_vitoria_fora = fora[(fora["fora"] == clubes.fora) & (fora["vencedor"] == clubes.fora)]["odd_fora"].mean()

odd_media_vitoria_fora = round((odd_media_vitoria_casa + odd_media_vitoria_fora) / 2, 2)

odd_media_vitoria_fora

1.28

Contando as goleadas

In [18911]:
goleadas_visitante_casa = fora[(fora["casa"] == clubes.fora) & (fora["gols_casa"] >= 4)].shape[0]
goleadas_visitante_fora = fora[(fora["fora"] == clubes.fora) & (fora["gols_fora"] >= 4)].shape[0]

goleadas_fora = goleadas_visitante_casa + goleadas_visitante_fora

goleadas_fora

5

% de goleadas aplicadas

In [18912]:
if jogos_fora == 0:
    prob_goleada_fora = 0
else:
    prob_goleada_fora = round((goleadas_fora / jogos_fora) * 100, 2)

prob_goleada_fora

35.71

Montando o DataFrame do time de fora

In [18913]:
dados_fora = {
    "Jogos": jogos_fora,
    "Vitórias": vitorias_fora,
    "Empates": empates_fora,
    "Derrotas": derrotas_fora,
    "Gols marcados": gols_marcados_fora,
    "Gols sofridos": gols_sofridos_fora,
    "Aproveitamento": aproveitamento_fora,
    "Over 2.5 gols": over_2_fora,
    "Probabilidade de over 2.5 gols": prob_over_2_fora,
    "Under 2.5 gols": under_2_fora,
    "Probabilidade de under 2.5 gols": prob_under_2_fora,
    "Odd média": odd_media_fora,
    "Odd média quando venceu": odd_media_vitoria_fora,
    "Goleadas": goleadas_fora,
    "Probabilidade de golear": prob_goleada_fora,
    "Goleadas como mandante": goleadas_mandante_fora,
    "Goleadas como visitante": goleadas_visitante_fora
}

In [18914]:
import numpy as np

# --- monta o DF com os dois dicionários (mantive sua lógica) ---
df_final_partidas = pd.DataFrame({
    clubes.casa: dados,
    clubes.fora: dados_fora
})

# --- lista de variáveis que devem ter 2 casas decimais (em minúsculas) ---
variaveis_float = {
    "aproveitamento",
    "probabilidade de over 2.5 gols",
    "probabilidade de under 2.5 gols",
    "odd média",
    "odd média quando venceu",
    "probabilidade de golear",
}

# --- função robusta de formatação ---
def formatar_valor(nome, valor):
    # normaliza o "nome" para comparação
    nome_norm = str(nome).strip().lower()

    # trata ausências
    if pd.isna(valor) or valor == "" or valor is None:
        # escolha 1: mostrar vazio para fácil leitura
        return ""
        # escolha 2 (alternativa): preserve como NaN
        # return np.nan

    # tenta converter para float (cobre ints, strings numéricas etc.)
    try:
        v = float(valor)
    except (ValueError, TypeError):
        # se não for numérico, devolve como string sem formatar
        return str(valor)

    # formatação conforme o tipo de métrica
    if nome_norm in variaveis_float:
        return f"{v:.2f}"
    else:
        # se for algo que deveria ser inteiro (contagens)
        # evita erro com NaN e converte inteiros "10.0" para "10"
        if np.isfinite(v) and float(v).is_integer():
            return f"{int(v)}"
        return f"{int(round(v))}"

# --- aplica formatação em todas as colunas ---
for col in df_final_partidas.columns:
    df_final_partidas[col] = [
        formatar_valor(nome, valor)
        for nome, valor in zip(df_final_partidas.index, df_final_partidas[col])
    ]

# --- transforma o índice em coluna ---
tabela = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [18915]:
tabela.style.hide(axis="index")

Informações,Al-Ettifaq,Al-Nassr
Jogos,11,14
Vitórias,4,12
Empates,4,1
Derrotas,3,1
Gols marcados,17,42
Gols sofridos,20,10
Aproveitamento,48.48,88.10
Over 2.5 gols,8,11
Probabilidade de over 2.5 gols,72.73,78.57
Under 2.5 gols,3,3


In [18916]:
caminho = r"C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx"

In [18917]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela.to_excel(writer, sheet_name="Partidas Atual", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Casa (últimos 10 jogos)

In [18918]:
casa = casa.sort_values("data", ascending=False).head(10)

casa

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
394,2025-12-25,2025,14771927,Saudita,Al-Riyadh,0,2,Al-Ettifaq,3.20,3.40,2.10,Al-Ettifaq,2
392,2025-11-22,2025,14771897,Saudita,Al-Ettifaq,3,2,Al-Fayha,2.80,3.30,2.38,Al-Ettifaq,5
390,2025-11-08,2025,14771811,Saudita,Al-Shabab,1,1,Al-Ettifaq,1.95,3.50,3.40,Empate,2
388,2025-10-31,2025,14771807,Saudita,Al-Ettifaq,2,2,Al-Hazem,2.05,3.50,3.40,Empate,4
385,2025-10-24,2025,14429574,Saudita,Al-Fateh,2,1,Al-Ettifaq,2.40,3.60,2.75,Al-Fateh,3
384,2025-10-18,2025,14429564,Saudita,Al-Ettifaq,0,5,Al-Hilal,5.75,4.75,1.42,Al-Hilal,5
382,2025-09-26,2025,14429557,Saudita,Damac,1,3,Al-Ettifaq,2.90,3.10,2.30,Al-Ettifaq,4
379,2025-09-21,2025,13975527,Copa da Arábia,Al-Batin,2,2,Al-Ettifaq,18.00,7.30,1.13,Empate,4
377,2025-09-18,2025,14429548,Saudita,Al-Taawoun,4,1,Al-Ettifaq,1.95,3.40,3.60,Al-Taawoun,5
346,2025-09-12,2025,14429537,Saudita,Al-Ettifaq,0,0,Al-Ahli,5.00,4.20,1.48,Empate,0


Contando os jogos

In [18919]:
jogos_casa = casa.shape[0]

jogos_casa

10

Contando as vitórias

In [18920]:
vitorias = casa[casa["vencedor"] == clubes.casa]

vitorias_casa = vitorias.shape[0]

vitorias_casa

3

Contando os empates

In [18921]:
empates = casa[casa["vencedor"] == "Empate"]

empates_casa = empates.shape[0]

empates_casa

4

Contando as derrotas

In [18922]:
derrotas = casa[(casa["vencedor"] != clubes.casa) & (casa["vencedor"] != "Empate")]

derrotas_casa = derrotas.shape[0]

derrotas_casa

3

Gols marcados

In [18923]:
gols_casa = casa.loc[casa["casa"] == clubes.casa, "gols_casa"].sum()
gols_fora = casa.loc[casa["fora"] == clubes.casa, "gols_fora"].sum()

gols_marcados_casa = gols_casa + gols_fora

gols_marcados_casa

C:\Users\stefa\AppData\Roaming\Python\Python312\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


15

Gols sofridos

In [18924]:
gols_sofridos_casa = casa.loc[casa["casa"] == clubes.casa, "gols_fora"].sum()
gols_sofridos_fora = casa.loc[casa["fora"] == clubes.casa, "gols_casa"].sum()

gols_sofridos_casa = gols_sofridos_casa + gols_sofridos_fora

gols_sofridos_casa

19

Aproveitamento

In [18925]:
aproveitamento_casa = round((vitorias_casa * 3 + empates_casa) / (jogos_casa * 3) * 100, 2)

aproveitamento_casa

43.33

Over 2.5 gols

In [18926]:
over_2_casa = casa[casa["total_gols"] >= 3].shape[0]

over_2_casa

7

% de jogos com 3 ou mais gols

In [18927]:
prob_over_2_casa = round((over_2_casa / jogos_casa) * 100, 2)

prob_over_2_casa

70.0

Under 2.5 gols

In [18928]:
under_2_casa = casa[casa["total_gols"] < 3].shape[0]

under_2_casa

3

% de jogos com menos de 3 gols

In [18929]:
prob_under_2_casa = round((under_2_casa / jogos_casa) * 100, 2)

prob_under_2_casa

30.0

Odd média

In [18930]:
odd_media_casa = casa[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_fora = casa[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_casa = round((odd_media_casa + odd_media_fora) / 2, 2)

odd_media_casa

3.22

Odd média quando venceu

In [18931]:
odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_vitoria_casa = round(odd_media_vitoria_casa + odd_media_vitoria_fora / 2, 2)

odd_media_vitoria_casa

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\524038383.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\524038383.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()


3.9

Contando as goleadas

In [18932]:
goleadas_mandante_casa = casa[(casa["casa"] == clubes.casa) & (casa["gols_casa"] >= 4)].shape[0]
goleadas_mandante_fora = casa[(casa["fora"] == clubes.casa) & (casa["gols_fora"] >= 4)].shape[0]

goleadas_casa = goleadas_mandante_casa + goleadas_mandante_fora

goleadas_casa

0

% de goleadas aplicadas

In [18933]:
prob_goleada_casa = round((goleadas_casa / jogos_casa) * 100, 2)

prob_goleada_casa

0.0

Montando o DataFrame do time da casa

In [18934]:
dados = {
    "Jogos": jogos_casa,
    "Vitórias": vitorias_casa,
    "Empates": empates_casa,
    "Derrotas": derrotas_casa,
    "Gols marcados": gols_marcados_casa,
    "Gols sofridos": gols_sofridos_casa,
    "Aproveitamento": aproveitamento_casa,
    "Over 2.5 gols": over_2_casa,
    "Probabilidade de over 2.5 gols": prob_over_2_casa,
    "Under 2.5 gols": under_2_casa,
    "Probabilidade de under 2.5 gols": prob_under_2_casa,
    "Odd média": odd_media_casa,
    "Odd média quando venceu": odd_media_vitoria_casa,
    "Goleadas": goleadas_casa,
    "Probabilidade de golear": prob_goleada_casa,
    "Goleadas como mandante": goleadas_mandante_casa,
    "Goleadas como visitante": goleadas_mandante_fora
}

In [18935]:
ordem = [
    "Jogos", "Vitórias", "Empates", "Derrotas",
    "Gols marcados", "Gols sofridos",
    "Aproveitamento",
    "Over 2.5 gols", "Probabilidade de over 2.5 gols",
    "Under 2.5 gols", "Probabilidade de under 2.5 gols",
    "Odd média", "Odd média quando venceu",
    "Goleadas", "Probabilidade de golear",
]

faltando = [k for k in ordem if k not in dados]
if faltando:
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")


dados_ordenados = {k: dados[k] for k in ordem}
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=[clubes.casa]
)


variaveis_float = {
    "aproveitamento",
    "probabilidade de over 2.5 gols",
    "probabilidade de under 2.5 gols",
    "odd média",
    "odd média quando venceu",
    "probabilidade de golear",
}

def _fmt(nome, valor):
    if str(nome).lower() in variaveis_float:
        return f"{float(valor):.2f}"
    else:
        return f"{int(valor)}"

df_final_partidas[clubes.casa] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[clubes.casa])
]

# Índice vira coluna "Informações"
tabela_casa = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [18936]:
tabela_casa.style.hide(axis="index")

Informações,Al-Ettifaq
Jogos,10
Vitórias,3
Empates,4
Derrotas,3
Gols marcados,15
Gols sofridos,19
Aproveitamento,43.33
Over 2.5 gols,7
Probabilidade de over 2.5 gols,70.00
Under 2.5 gols,3


### Fora (últimos 10 jogos)

In [18937]:
fora = df[(df["casa"] == clubes.fora) | (df["fora"] == clubes.fora)]

Selecionando a temporada atual

In [18938]:
fora = fora[fora["temporada"] == "2025"]

In [18939]:
fora = fora.sort_values("data", ascending=False).head(10)

fora

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
395,2025-12-27,2025,14771921,Saudita,Al-Nassr,3,0,Al-Okhdood,1.14,8.0,15.00,Al-Nassr,3
393,2025-11-23,2025,14771901,Saudita,Al-Nassr,4,1,Al-Khaleej,1.17,7.0,10.00,Al-Nassr,5
391,2025-11-08,2025,14771806,Saudita,Neom,1,3,Al-Nassr,6.25,5.0,1.33,Al-Nassr,4
389,2025-11-01,2025,14833584,Saudita,Al-Nassr,2,1,Al-Fayha,1.17,7.0,17.00,Al-Nassr,3
387,2025-10-28,2025,14768545,Copa da Arábia,Al-Nassr,1,2,Al-Ittihad,1.42,4.5,5.50,Al-Ittihad,3
386,2025-10-25,2025,14429580,Saudita,Al-Hazem,0,2,Al-Nassr,11.00,8.5,1.14,Al-Nassr,2
383,2025-10-18,2025,14429567,Saudita,Al-Nassr,5,1,Al-Fateh,1.11,8.5,15.00,Al-Nassr,6
381,2025-09-26,2025,14429558,Saudita,Al-Ittihad,0,2,Al-Nassr,3.40,4.5,1.75,Al-Nassr,2
380,2025-09-23,2025,13975525,Copa da Arábia,Jeddah,0,4,Al-Nassr,23.00,7.5,1.11,Al-Nassr,4
378,2025-09-20,2025,14429555,Saudita,Al-Nassr,5,1,Al-Riyadh,1.10,11.0,17.00,Al-Nassr,6


Contando os jogos

In [18940]:
jogos_fora = fora.shape[0]

jogos_fora

10

Contando as vitórias

In [18941]:
vitorias = fora[fora["vencedor"] == clubes.fora]

vitorias_fora = vitorias.shape[0]

vitorias_fora

9

Contando os empates

In [18942]:
empates = fora[fora["vencedor"] == "Empate"]

empates_fora = empates.shape[0]

empates_fora

0

Contando as derrotas

In [18943]:
derrotas = fora[(fora["vencedor"] != clubes.fora) & (fora["vencedor"] != "Empate")]

derrotas_fora = derrotas.shape[0]

derrotas_fora

1

Gols marcados

In [18944]:
gols_casa = fora.loc[fora["casa"] == clubes.fora, "gols_casa"].sum()
gols_fora = fora.loc[fora["fora"] == clubes.fora, "gols_fora"].sum()

gols_marcados_fora = gols_casa + gols_fora

gols_marcados_fora

31

Gols sofridos

In [18945]:
gols_sofridos_casa = fora.loc[fora["casa"] == clubes.fora, "gols_fora"].sum()
gols_sofridos_fora = fora.loc[fora["fora"] == clubes.fora, "gols_casa"].sum()

gols_sofridos_fora = gols_sofridos_casa + gols_sofridos_fora

gols_sofridos_fora

7

Aproveitamento

In [18946]:
if jogos_fora == 0:
    aproveitamento_fora = 0
else:
    aproveitamento_fora = round((vitorias_fora * 3 + empates_fora) / (jogos_fora * 3) * 100, 2)

aproveitamento_fora

90.0

Over 2.5 gols

In [18947]:
over_2_fora = fora[fora["total_gols"] >= 3].shape[0]

over_2_fora

8

% de jogos com 3 ou mais gols

In [18948]:
if jogos_fora == 0:
    prob_over_2_fora = 0
else:
    prob_over_2_fora = round((over_2_fora / jogos_fora) * 100, 2)

prob_over_2_fora

80.0

Under 2.5 gols

In [18949]:
under_2_fora = fora[fora["total_gols"] < 3].shape[0]

under_2_fora

2

% de jogos com menos de 3 gols

In [18950]:
if jogos_fora == 0:
    prob_under_2_fora = 0
else:
    prob_under_2_fora = round((under_2_fora / jogos_fora) * 100, 2)

prob_under_2_fora

20.0

Odd média

In [18951]:
odd_media_casa = fora[fora["casa"] == clubes.fora]["odd_casa"].mean()
odd_media_fora = fora[fora["fora"] == clubes.fora]["odd_fora"].mean()

odd_media_fora = round((odd_media_casa + odd_media_fora) / 2, 2)

odd_media_fora

1.26

Odd média quando venceu

In [18952]:
odd_media_vitoria_casa = fora[(fora["casa"] == clubes.fora) & (fora["vencedor"] == clubes.fora)]["odd_casa"].mean()
odd_media_vitoria_fora = fora[(fora["fora"] == clubes.fora) & (fora["vencedor"] == clubes.fora)]["odd_fora"].mean()

odd_media_vitoria_fora = round((odd_media_vitoria_casa + odd_media_vitoria_fora) / 2, 2)

odd_media_vitoria_fora

1.24

Contando as goleadas

In [18953]:
goleadas_mandante_fora = fora[(fora["casa"] == clubes.fora) & (fora["gols_casa"] >= 4)].shape[0]
goleadas_visitante_fora = fora[(fora["fora"] == clubes.fora) & (fora["gols_fora"] >= 4)].shape[0]

goleadas_fora = goleadas_mandante_fora + goleadas_visitante_fora

goleadas_fora

4

% de goleadas aplicadas

In [18954]:
if jogos_fora == 0:
    prob_goleada_fora = 0
else:
    prob_goleada_fora = round((goleadas_fora / jogos_fora) * 100, 2)

prob_goleada_fora

40.0

Montando o DataFrame do time de fora

In [18955]:
dados_fora = {
    "Jogos": jogos_fora,
    "Vitórias": vitorias_fora,
    "Empates": empates_fora,
    "Derrotas": derrotas_fora,
    "Gols marcados": gols_marcados_fora,
    "Gols sofridos": gols_sofridos_fora,
    "Aproveitamento": aproveitamento_fora,
    "Over 2.5 gols": over_2_fora,
    "Probabilidade de over 2.5 gols": prob_over_2_fora,
    "Under 2.5 gols": under_2_fora,
    "Probabilidade de under 2.5 gols": prob_under_2_fora,
    "Odd média": odd_media_fora,
    "Odd média quando venceu": odd_media_vitoria_fora,
    "Goleadas": goleadas_fora,
    "Probabilidade de golear": prob_goleada_fora,
    "Goleadas como mandante": goleadas_mandante_fora,
    "Goleadas como visitante": goleadas_visitante_fora
}

In [18956]:
import numpy as np

# --- monta o DF com os dois dicionários (mantive sua lógica) ---
df_final_partidas = pd.DataFrame({
    clubes.casa: dados,
    clubes.fora: dados_fora
})

# --- lista de variáveis que devem ter 2 casas decimais (em minúsculas) ---
variaveis_float = {
    "aproveitamento",
    "probabilidade de over 2.5 gols",
    "probabilidade de under 2.5 gols",
    "odd média",
    "odd média quando venceu",
    "probabilidade de golear",
}

# --- função robusta de formatação ---
def formatar_valor(nome, valor):
    # normaliza o "nome" para comparação
    nome_norm = str(nome).strip().lower()

    # trata ausências
    if pd.isna(valor) or valor == "" or valor is None:
        # escolha 1: mostrar vazio para fácil leitura
        return ""
        # escolha 2 (alternativa): preserve como NaN
        # return np.nan

    # tenta converter para float (cobre ints, strings numéricas etc.)
    try:
        v = float(valor)
    except (ValueError, TypeError):
        # se não for numérico, devolve como string sem formatar
        return str(valor)

    # formatação conforme o tipo de métrica
    if nome_norm in variaveis_float:
        return f"{v:.2f}"
    else:
        # se for algo que deveria ser inteiro (contagens)
        # evita erro com NaN e converte inteiros "10.0" para "10"
        if np.isfinite(v) and float(v).is_integer():
            return f"{int(v)}"
        # se vier um 3.7 em métrica "inteira", escolha:
        #  - truncar: int(v)  -> "3"
        #  - arredondar: round(v) -> "4"
        # aqui vou arredondar para evitar truncar silenciosamente
        return f"{int(round(v))}"

# --- aplica formatação em todas as colunas ---
for col in df_final_partidas.columns:
    df_final_partidas[col] = [
        formatar_valor(nome, valor)
        for nome, valor in zip(df_final_partidas.index, df_final_partidas[col])
    ]

# --- transforma o índice em coluna ---
tabela_recente = df_final_partidas.reset_index().rename(columns={"index": "Informações"})


In [18957]:
tabela_recente.style.hide(axis="index")

Informações,Al-Ettifaq,Al-Nassr
Jogos,10,10
Vitórias,3,9
Empates,4,0
Derrotas,3,1
Gols marcados,15,31
Gols sofridos,19,7
Aproveitamento,43.33,90.00
Over 2.5 gols,7,8
Probabilidade de over 2.5 gols,70.00,80.00
Under 2.5 gols,3,2


Salvando o arquivo

In [18958]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_recente.to_excel(writer, sheet_name="Desempenho 10 Jogos", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Clean sheets e clean goals (10 jogos)

In [18959]:
def clean_stats_do_clube(df: pd.DataFrame, clube: str) -> tuple[int, int]:

    # Mantém só as partidas do clube (seja mandante ou visitante)
    m = (df["casa"] == clube) | (df["fora"] == clube)
    jogos = df.loc[m].copy()

    if jogos.empty:
        return 0, 0

    # Gols a favor/contra de acordo com o lado em cada jogo
    gols_favor = np.where(jogos["casa"] == clube, jogos["gols_casa"], jogos["gols_fora"])
    gols_contra = np.where(jogos["casa"] == clube, jogos["gols_fora"], jogos["gols_casa"])

    # Protege contra NaN
    gols_favor = pd.Series(gols_favor).fillna(0)
    gols_contra = pd.Series(gols_contra).fillna(0)

    clean_sheets = int((gols_contra == 0).sum())  # não sofreu
    clean_goals  = int((gols_favor  == 0).sum())  # não marcou
    return clean_sheets, clean_goals


def montar_df_clean_unificado(df_casa: pd.DataFrame, df_fora: pd.DataFrame, clubes) -> pd.DataFrame:

    # Lê nomes dos clubes do objeto/dict
    clube_casa = getattr(clubes, "casa", None) if not isinstance(clubes, dict) else clubes.get("casa")
    clube_fora = getattr(clubes, "fora", None) if not isinstance(clubes, dict) else clubes.get("fora")

    # Calcula para cada clube, considerando que cada DF já contém os 10 jogos daquele clube
    cs_casa, cg_casa = clean_stats_do_clube(df_casa, clube_casa)
    cs_fora, cg_fora = clean_stats_do_clube(df_fora, clube_fora)

    # Deixa 'clube' como coluna normal (não índice) para evitar cabeçalho em 2 linhas
    resultado = pd.DataFrame(
        [
            {"clube": clube_casa, "clean_sheets": cs_casa, "clean_goals": cg_casa},
            {"clube": clube_fora, "clean_sheets": cs_fora, "clean_goals": cg_fora},
        ],
        columns=["clube", "clean_sheets", "clean_goals"],  # força a ordem
    )

    return resultado

In [18960]:
df_cleans = montar_df_clean_unificado(casa, fora, clubes)

df_cleans

,clube,clean_sheets,clean_goals
0,Al-Ettifaq,2,2
1,Al-Nassr,4,0


### Força no mando

#### Carregando a consulta novamente

In [18961]:
conn = mysql.connector.connect(**config)

In [18962]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols
FROM
	partidas
WHERE
	(casa = "{casa}" OR fora = "{casa}" OR casa = "{clubes.fora}" OR fora = "{clubes.fora}");
"""

In [18963]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [18964]:
conn.close()

In [18965]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2024-08-22,2024/2025,12545108,Saudita,Al-Nassr,1,1,Al-Raed,1.14,8.0,11.00,Empate,2
1,2024-08-27,2024/2025,12618489,Saudita,Al-Fayha,1,4,Al-Nassr,12.00,9.5,1.11,Al-Nassr,5
2,2024-09-13,2024/2025,12625053,Saudita,Al-Nassr,1,1,Al-Ahli,1.70,3.9,3.90,Empate,2
3,2024-09-20,2024/2025,12625039,Saudita,Al-Ettifaq,0,3,Al-Nassr,4.50,4.0,1.60,Al-Nassr,3
4,2024-09-27,2024/2025,12625035,Saudita,Al-Nassr,2,0,Al-Wehda,1.14,7.0,13.00,Al-Nassr,2


#### Carregando o time da casa

In [18966]:
casa = df.copy()

Selecionando o time da casa e filtrando os últimos 10 jogos

In [18967]:
casa = casa[casa["casa"] == clubes.casa]

In [18968]:
casa = casa.sort_values("data", ascending=False).head(10)

In [18969]:
def func_desempenho_casa(casa_df):
    gc = casa_df["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)
    gf = casa_df["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)

    jogos     = int(len(casa_df))
    vitorias  = int((gc > gf).sum())
    empates   = int((gc == gf).sum())
    derrotas  = int((gc < gf).sum())
    marcados  = int(gc.sum())
    sofridos  = int(gf.sum())

    # Odds
    odd_casa = casa_df["odd_casa"].apply(pd.to_numeric, errors="coerce")
    odd_media = odd_casa.mean(skipna=True)
    odd_media_vitoria = odd_casa[gc > gf].mean(skipna=True)
    # evita NaN quando não houver vitórias
    odd_media_vitoria = 0.0 if pd.isna(odd_media_vitoria) else odd_media_vitoria

    denom = 30 if jogos >= 10 else max(jogos * 3, 1)
    aproveitamento = ((vitorias * 3 + empates) / denom) * 100          # 0–100
    media_marcados = (marcados / jogos) if jogos else 0.0
    media_sofridos = (sofridos / jogos) if jogos else 0.0

    clean_sheets = int((gf == 0).sum())
    clean_goals  = int((gc == 0).sum())

    linhas = [
        {"informacao": "Jogos",                        "dados": jogos},
        {"informacao": "Vitórias",                     "dados": vitorias},
        {"informacao": "Empates",                      "dados": empates},
        {"informacao": "Derrotas",                     "dados": derrotas},
        {"informacao": "Gols marcados",                "dados": marcados},
        {"informacao": "Gols sofridos",                "dados": sofridos},
        {"informacao": "Aproveitamento",               "dados": round(aproveitamento, 2)},
        {"informacao": "Média de gols marcados",       "dados": round(media_marcados, 2)},
        {"informacao": "Média de gols sofridos",       "dados": round(media_sofridos, 2)},
        {"informacao": "Clean sheets",                 "dados": clean_sheets},
        {"informacao": "Clean goals",                  "dados": clean_goals},
        {"informacao": "Odd média",                    "dados": round(float(odd_media) if not pd.isna(odd_media) else 0.0, 2)},
        {"informacao": "Odd média quando venceu",      "dados": round(float(odd_media_vitoria), 2)},
    ]

    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Inteiros obrigatórios
    int_infos = {
        "Jogos","Vitórias","Empates","Derrotas",
        "Gols marcados","Gols sofridos","Clean sheets","Clean goals"
    }
    mask_int = df["informacao"].isin(int_infos)
    df.loc[mask_int, "dados"] = df.loc[mask_int, "dados"].astype(int)

    # Formatação para exibição (2 casas só nos floats)
    float_infos = {
        "Aproveitamento", "Média de gols marcados", "Média de gols sofridos",
        "Odd média", "Odd média quando venceu"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )

    return df

In [18970]:
desempenho_casa = func_desempenho_casa(casa)

desempenho_casa = desempenho_casa[["informacao", "dados_fmt"]]

desempenho_casa = desempenho_casa.rename(columns={"dados_fmt": "dados"})

desempenho_casa

,informacao,dados
0,Jogos,5
1,Vitórias,1
2,Empates,3
3,Derrotas,1
4,Gols marcados,6
5,Gols sofridos,8
6,Aproveitamento,40.00
7,Média de gols marcados,1.20
8,Média de gols sofridos,1.60
9,Clean sheets,0


#### Carregando o time de fora

In [18971]:
fora = df.copy()

Selecionando o time da casa e filtrando os últimos 10 jogos

In [18972]:
fora = fora[fora["fora"] == clubes.fora]

In [18973]:
fora = fora.sort_values("data", ascending=False).head(10)

In [18974]:
def func_desempenho_fora(fora_df):
    # gols do visitante = a favor; gols do mandante = contra
    gf = fora_df["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)  # gols a favor
    gc = fora_df["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)  # gols contra

    jogos     = int(len(fora_df))
    vitorias  = int((gf > gc).sum())
    empates   = int((gf == gc).sum())
    derrotas  = int((gf < gc).sum())
    marcados  = int(gf.sum())
    sofridos  = int(gc.sum())

    # Odds do visitante
    odd_fora = fora_df["odd_fora"].apply(pd.to_numeric, errors="coerce")
    odd_media = odd_fora.mean(skipna=True)
    odd_media_vitoria = odd_fora[gf > gc].mean(skipna=True)
    odd_media_vitoria = 0.0 if pd.isna(odd_media_vitoria) else odd_media_vitoria

    denom = 30 if jogos >= 10 else max(jogos * 3, 1)
    aproveitamento = ((vitorias * 3 + empates) / denom) * 100           # 0–100
    media_marcados = (marcados / jogos) if jogos else 0.0               # float
    media_sofridos = (sofridos / jogos) if jogos else 0.0               # float

    clean_sheets = int((gc == 0).sum())  # não sofreu gols (mandante 0)
    clean_goals  = int((gf == 0).sum())  # não marcou gols (visitante 0)

    linhas = [
        {"informacao": "Jogos",                        "dados": jogos},
        {"informacao": "Vitórias",                     "dados": vitorias},
        {"informacao": "Empates",                      "dados": empates},
        {"informacao": "Derrotas",                     "dados": derrotas},
        {"informacao": "Gols marcados",                "dados": marcados},
        {"informacao": "Gols sofridos",                "dados": sofridos},
        {"informacao": "Aproveitamento",               "dados": round(aproveitamento, 2)},
        {"informacao": "Média de gols marcados",       "dados": round(media_marcados, 2)},
        {"informacao": "Média de gols sofridos",       "dados": round(media_sofridos, 2)},
        {"informacao": "Clean sheets",                 "dados": clean_sheets},
        {"informacao": "Clean goals",                  "dados": clean_goals},
        {"informacao": "Odd média",                    "dados": round(float(odd_media) if not pd.isna(odd_media) else 0.0, 2)},
        {"informacao": "Odd média quando venceu",      "dados": round(float(odd_media_vitoria), 2)},
    ]

    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Linhas que DEVEM ser inteiras
    int_infos = {
        "Jogos","Vitórias","Empates","Derrotas",
        "Gols marcados","Gols sofridos","Clean sheets","Clean goals"
    }
    mask_int = df["informacao"].isin(int_infos)
    df.loc[mask_int, "dados"] = df.loc[mask_int, "dados"].astype(int)

    # Coluna de exibição: inteiros sem casas; floats com 2 casas
    float_infos = {
        "Aproveitamento", "Média de gols marcados", "Média de gols sofridos",
        "Odd média", "Odd média quando venceu"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )
    return df

In [18975]:
desempenho_fora = func_desempenho_fora(fora)

desempenho_fora = desempenho_fora[["informacao", "dados_fmt"]]

desempenho_fora = desempenho_fora.rename(columns={"dados_fmt": "dados"})

desempenho_fora

,informacao,dados
0,Jogos,10
1,Vitórias,8
2,Empates,0
3,Derrotas,2
4,Gols marcados,35
5,Gols sofridos,9
6,Aproveitamento,80.00
7,Média de gols marcados,3.50
8,Média de gols sofridos,0.90
9,Clean sheets,5


### Confronto direto

#### Carregando a consulta

In [18976]:
conn = mysql.connector.connect(**config)

In [18977]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols
FROM
	partidas
WHERE
	(casa = "{casa}" OR fora = "{casa}")
    AND (casa = "{clubes.fora}" OR fora = "{clubes.fora}");
"""

In [18978]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [18979]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols


#### H2H - Geral

In [18980]:
confronto_direto = df.copy()

In [18981]:
def func_desempenho_confronto_direto(confronto_direto, clubes):
    # Gols como numérico
    gc = confronto_direto["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)
    gf = confronto_direto["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)

    # Máscaras de quem é quem em cada linha
    m_casa_home = confronto_direto["casa"] == clubes.casa
    m_casa_away = confronto_direto["fora"] == clubes.casa
    m_fora_home = confronto_direto["casa"] == clubes.fora
    m_fora_away = confronto_direto["fora"] == clubes.fora

    jogos = int(len(confronto_direto))

    # Vitórias corretas (independe de onde jogou)
    vitorias_casa = int(((m_casa_home & (gc > gf)) | (m_casa_away & (gf > gc))).sum())
    empates       = int((gc == gf).sum())
    vitorias_fora = int(((m_fora_home & (gc > gf)) | (m_fora_away & (gf > gc))).sum())

    # Gols por clube (soma do lado correspondente em cada linha)
    gols_do_casa = int(gc[m_casa_home].sum() + gf[m_casa_away].sum())
    gols_do_fora = int(gc[m_fora_home].sum() + gf[m_fora_away].sum())

    # Média de gols do duelo
    if "total_gols" in confronto_direto.columns:
        total_gols = confronto_direto["total_gols"].apply(pd.to_numeric, errors="coerce")
    else:
        total_gols = gc + gf
    media_gols_duelo = round(total_gols.mean(skipna=True) if jogos else 0.0, 2)

    # ===== Odds (condicionadas ao lado) =====
    odd_casa_col = confronto_direto["odd_casa"].apply(pd.to_numeric, errors="coerce")
    odd_fora_col = confronto_direto["odd_fora"].apply(pd.to_numeric, errors="coerce")

    # Odd média do clubes.casa: quando ele é mandante usa odd_casa; quando visitante usa odd_fora
    odds_clubes_casa = pd.concat([
        odd_casa_col[m_casa_home],
        odd_fora_col[m_casa_away]
    ])
    odd_media_clubes_casa = float(odds_clubes_casa.mean(skipna=True) or 0.0)

    # Odd média do clubes.fora: análogo
    odds_clubes_fora = pd.concat([
        odd_casa_col[m_fora_home],
        odd_fora_col[m_fora_away]
    ])
    odd_media_clubes_fora = float(odds_clubes_fora.mean(skipna=True) or 0.0)

    # Odd média do vencedor (ignora empates)
    home_win = gc > gf
    away_win = gf > gc
    odds_vencedor = pd.concat([
        odd_casa_col[home_win],
        odd_fora_col[away_win]
    ])
    odd_media_vencedor = float(odds_vencedor.mean(skipna=True) or 0.0)

    # ===== Monta saída =====
    linhas = [
        {"informacao": "Jogos",                                 "dados": jogos},
        {"informacao": f"Vitórias do {clubes.casa}",            "dados": vitorias_casa},
        {"informacao": "Empates",                               "dados": empates},
        {"informacao": f"Vitórias do {clubes.fora}",            "dados": vitorias_fora},
        {"informacao": f"Gols do {clubes.casa}",                "dados": gols_do_casa},
        {"informacao": f"Gols do {clubes.fora}",                "dados": gols_do_fora},
        {"informacao": "Média de gols do duelo",                "dados": media_gols_duelo},
        {"informacao": f"Odd média do {clubes.casa}",           "dados": round(odd_media_clubes_casa, 2)},
        {"informacao": f"Odd média do {clubes.fora}",           "dados": round(odd_media_clubes_fora, 2)},
        {"informacao": "Odd média do vencedor",                 "dados": round(odd_media_vencedor, 2)},
    ]
    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Inteiros garantidos
    int_infos = {
        "Jogos",
        f"Vitórias do {clubes.casa}", "Empates", f"Vitórias do {clubes.fora}",
        f"Gols do {clubes.casa}", f"Gols do {clubes.fora}"
    }
    df.loc[df["informacao"].isin(int_infos), "dados"] = df.loc[df["informacao"].isin(int_infos), "dados"].astype(int)

    # Formatação de exibição: 2 casas para médias/odds
    float_infos = {
        "Média de gols do duelo",
        f"Odd média do {clubes.casa}",
        f"Odd média do {clubes.fora}",
        "Odd média do vencedor"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )

    return df

In [18982]:
desempenho_confronto = func_desempenho_confronto_direto(confronto_direto, clubes)

desempenho_confronto = desempenho_confronto[["informacao", "dados_fmt"]]

desempenho_confronto = desempenho_confronto.rename(columns={"dados_fmt": "dados"})

desempenho_confronto

,informacao,dados
0,Jogos,0
1,Vitórias do Al-Ettifaq,0
2,Empates,0
3,Vitórias do Al-Nassr,0
4,Gols do Al-Ettifaq,0
5,Gols do Al-Nassr,0
6,Média de gols do duelo,0.00
7,Odd média do Al-Ettifaq,nan
8,Odd média do Al-Nassr,nan
9,Odd média do vencedor,nan


#### H2H - últimos 10 jogos

In [18983]:
confronto_recente = df.copy()

In [18984]:
confronto_recente = confronto_recente.sort_values("data", ascending=False).head(10)

In [18985]:
def func_desempenho_confronto_recente(confronto_recente, clubes):
    # ---- Converte gols para numérico ----
    gc = confronto_recente["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)
    gf = confronto_recente["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)

    # ---- Máscaras: posição de cada clube em cada jogo ----
    m_casa_home = confronto_recente["casa"] == clubes.casa
    m_casa_away = confronto_recente["fora"] == clubes.casa
    m_fora_home = confronto_recente["casa"] == clubes.fora
    m_fora_away = confronto_recente["fora"] == clubes.fora

    jogos = int(len(confronto_recente))

    # ---- Vitórias/Empates (independe de onde jogou) ----
    vitorias_casa = int(((m_casa_home & (gc > gf)) | (m_casa_away & (gf > gc))).sum())
    empates       = int((gc == gf).sum())
    vitorias_fora = int(((m_fora_home & (gc > gf)) | (m_fora_away & (gf > gc))).sum())

    # ---- Gols por clube (lado correto em cada linha) ----
    gols_do_casa = int(gc[m_casa_home].sum() + gf[m_casa_away].sum())
    gols_do_fora = int(gc[m_fora_home].sum() + gf[m_fora_away].sum())

    # ---- Média de gols do duelo ----
    if "total_gols" in confronto_recente.columns:
        total_gols = confronto_recente["total_gols"].apply(pd.to_numeric, errors="coerce")
    else:
        total_gols = gc + gf
    media_gols_duelo = round(total_gols.mean(skipna=True) if jogos else 0.0, 2)

    # ---- Odds (usando a odd do lado correto em cada jogo) ----
    odd_casa_col = confronto_recente["odd_casa"].apply(pd.to_numeric, errors="coerce")
    odd_fora_col = confronto_recente["odd_fora"].apply(pd.to_numeric, errors="coerce")

    # Odd média do clubes.casa
    odds_clubes_casa = pd.concat([odd_casa_col[m_casa_home], odd_fora_col[m_casa_away]])
    odd_media_clubes_casa = float(odds_clubes_casa.mean(skipna=True) or 0.0)

    # Odd média do clubes.fora
    odds_clubes_fora = pd.concat([odd_casa_col[m_fora_home], odd_fora_col[m_fora_away]])
    odd_media_clubes_fora = float(odds_clubes_fora.mean(skipna=True) or 0.0)

    # Odd média do vencedor (ignora empates)
    home_win = gc > gf
    away_win = gf > gc
    odds_vencedor = pd.concat([odd_casa_col[home_win], odd_fora_col[away_win]])
    odd_media_vencedor = float(odds_vencedor.mean(skipna=True) or 0.0)

    # ---- Monta saída ----
    linhas = [
        {"informacao": "Jogos",                                 "dados": jogos},
        {"informacao": f"Vitórias do {clubes.casa}",            "dados": vitorias_casa},
        {"informacao": "Empates",                               "dados": empates},
        {"informacao": f"Vitórias do {clubes.fora}",            "dados": vitorias_fora},
        {"informacao": f"Gols do {clubes.casa}",                "dados": gols_do_casa},
        {"informacao": f"Gols do {clubes.fora}",                "dados": gols_do_fora},
        {"informacao": "Média de gols do duelo",                "dados": media_gols_duelo},
        {"informacao": f"Odd média do {clubes.casa}",           "dados": round(odd_media_clubes_casa, 2)},
        {"informacao": f"Odd média do {clubes.fora}",           "dados": round(odd_media_clubes_fora, 2)},
        {"informacao": "Odd média do vencedor",                 "dados": round(odd_media_vencedor, 2)},
    ]
    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Inteiros garantidos
    int_infos = {
        "Jogos",
        f"Vitórias do {clubes.casa}", "Empates", f"Vitórias do {clubes.fora}",
        f"Gols do {clubes.casa}", f"Gols do {clubes.fora}"
    }
    df.loc[df["informacao"].isin(int_infos), "dados"] = df.loc[df["informacao"].isin(int_infos), "dados"].astype(int)

    # Formatação de exibição (2 casas para médias/odds)
    float_infos = {
        "Média de gols do duelo",
        f"Odd média do {clubes.casa}",
        f"Odd média do {clubes.fora}",
        "Odd média do vencedor"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )
    return df

In [18986]:
historico_recente = func_desempenho_confronto_recente(confronto_recente, clubes)

historico_recente = historico_recente[["informacao", "dados_fmt"]]

historico_recente = historico_recente.rename(columns={"dados_fmt": "dados"})

historico_recente

,informacao,dados
0,Jogos,0
1,Vitórias do Al-Ettifaq,0
2,Empates,0
3,Vitórias do Al-Nassr,0
4,Gols do Al-Ettifaq,0
5,Gols do Al-Nassr,0
6,Média de gols do duelo,0.00
7,Odd média do Al-Ettifaq,nan
8,Odd média do Al-Nassr,nan
9,Odd média do vencedor,nan


## 1º Tempo

### Carregando a consulta

In [18987]:
conn = mysql.connector.connect(**config)

In [18988]:
query = f"""
SELECT
    p.data,
    t1.temporada,
    t1.clube,
    CASE
        WHEN t1.clube = p.casa THEN 'Casa'
        WHEN t1.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    t1.ball_possession,
    t1.expected_goals,
    t1.big_chances,
    t1.total_shots,
    t1.shots_on_target,
    t1.shots_inside_box,
    t1.hit_woodwork,
    t1.corner_kicks,
    t1.passes,
    t1.big_chances_scored,
    t1.accurate_passes,
    t1.through_balls,
    t1.final_third_entries,
    t1.fouled_in_final_third,
    t1.dispossessed,
    t1.duels,
    t1.yellow_cards,
    t1.red_cards
FROM
    1_tempo t1
    INNER JOIN partidas p ON t1.id_jogo = p.id_jogo
WHERE
    t1.clube IN ("{clubes.casa}", "{clubes.fora}");
"""

In [18989]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [18990]:
conn.close()

In [18991]:
df = df[df["temporada"] == "2025"]

In [18992]:
df.head()

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,through_balls,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
85,2025-08-19,2025,Al-Nassr,Casa,38%,0.0,2,10,2,8,0,3,147,1,127,0,20,2,0,55%,0,1
86,2025-08-23,2025,Al-Nassr,Casa,63%,0.0,3,7,3,6,0,3,187,1,154,0,26,1,3,57%,1,0
87,2025-08-28,2025,Al-Ettifaq,Casa,57%,0.0,0,5,2,3,0,1,229,0,205,0,22,0,3,38%,1,0
88,2025-08-29,2025,Al-Nassr,Fora,59%,0.0,0,7,4,4,0,2,244,0,216,0,31,0,3,76%,0,0
357,2025-09-12,2025,Al-Ettifaq,Casa,38%,0.0,1,2,0,1,0,2,140,0,116,1,17,1,2,46%,0,0


### Casa

Jogos

In [18993]:
# Filtrando apenas informações do time mandante
casa = df[df["clube"] == clubes.casa]

# Selecionando apenas os últimos 10 jogos
casa = casa.sort_values("data", ascending=False).head(10)

casa.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,through_balls,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
404,2025-12-25,2025,Al-Ettifaq,Fora,63%,0.0,0,5,2,2,1,0,328,0,299,0,35,1,2,48%,1,0
402,2025-11-22,2025,Al-Ettifaq,Casa,49%,0.0,2,4,3,2,0,0,220,2,184,0,13,2,3,48%,0,0
400,2025-11-08,2025,Al-Ettifaq,Fora,62%,0.0,1,3,1,2,0,2,228,0,190,0,28,1,2,50%,0,0


In [18994]:
jogos_casa = casa.shape[0]

jogos_casa

10

Posse de bola média

In [18995]:
casa["ball_possession"] = casa["ball_possession"].str.replace("%", "", regex=False)
casa["ball_possession"] = pd.to_numeric(casa["ball_possession"])

In [18996]:
posse_bola_casa = round(casa["ball_possession"].mean(), 2)

posse_bola_casa

51.0

XG Médio

In [18997]:
xg_medio_casa = casa.expected_goals.mean().round(2)

xg_medio_casa

0.0

XG Total

In [18998]:
xg_total_casa = casa.expected_goals.sum().round(2)

xg_total_casa

0.0

Grandes chances criadas

In [18999]:
grandes_chances_criadas_casa = casa.big_chances.sum()

grandes_chances_criadas_casa

13

Média de grandes chances por jogo

In [19000]:
grandes_chances_jogo_casa = round(grandes_chances_criadas_casa / jogos_casa, 2)

grandes_chances_jogo_casa

1.3

Grandes chances convertidas

In [19001]:
grandes_chances_convertidas_casa = casa.big_chances_scored.sum()

grandes_chances_convertidas_casa

5

Aproveitamento das grandes chances

In [19002]:
aproveitamento_grandes_chances_casa = round((grandes_chances_convertidas_casa / grandes_chances_criadas_casa) * 100, 2)

aproveitamento_grandes_chances_casa

38.46

Finalizações

In [19003]:
finalizacoes_casa = casa.total_shots.sum()

finalizacoes_casa

44

Média de finalizações por jogo

In [19004]:
finalizacoes_jogo_casa = round(finalizacoes_casa / jogos_casa, 2)

finalizacoes_jogo_casa

4.4

Finalizações certas

In [19005]:
finalizacoes_certas_casa = casa.shots_on_target.sum()

finalizacoes_certas_casa

19

Finaizações certas por jogo

In [19006]:
finalizacoes_certas_jogo_casa = round(finalizacoes_certas_casa / jogos_casa, 2)

finalizacoes_certas_jogo_casa

1.9

Aproveitamento das finalizações

In [19007]:
aproveitamento_finalizacoes_casa = round((finalizacoes_certas_casa / finalizacoes_casa) * 100, 2)

aproveitamento_finalizacoes_casa

43.18

Finalizações dentro da área

In [19008]:
finalizacoes_area_casa = casa.shots_inside_box.sum()

finalizacoes_area_casa

32

Finalizações dentro da área por jogo

In [19009]:
finalizacoes_area_jogo_casa = round(finalizacoes_area_casa / jogos_casa, 2)

finalizacoes_area_jogo_casa

3.2

Finalizações dentro da área em relação ao total de finalizações

In [19010]:
perc_finalizacoes_area_casa = round((finalizacoes_area_casa / finalizacoes_casa) * 100, 2)

perc_finalizacoes_area_casa

72.73

Bolas na trave

In [19011]:
bolas_trave_casa = casa.hit_woodwork.sum()

bolas_trave_casa

3

Escanteios

In [19012]:
escanteios_casa = casa.corner_kicks.sum()

escanteios_casa

20

Escanteios por jogo

In [19013]:
escanteios_jogo_casa = round(escanteios_casa / jogos_casa, 2)

escanteios_jogo_casa

2.0

Passes

In [19014]:
passes_casa = casa.passes.sum()

passes_casa

2148

Passes certos

In [19015]:
passes_certos_casa = casa.accurate_passes.sum()

passes_certos_casa

1815

Aproveitamento nos passes

In [19016]:
perc_passes_certos_casa = round((passes_certos_casa / passes_casa) * 100, 2)

perc_passes_certos_casa

84.5

Entradas no terço final

In [19017]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

237

Entradas no terço final por jogo

In [19018]:
emtradas_terco_final_jogo_casa = round(emtradas_terco_final_casa / jogos_casa, 2)

emtradas_terco_final_jogo_casa

23.7

Faltas sofridas no terço final

In [19019]:
faltas_sofridas_terco_final_casa = casa.fouled_in_final_third.sum()

faltas_sofridas_terco_final_casa

11

Perda da posse de bola

In [19020]:
perda_bola_casa = casa.dispossessed.sum()

perda_bola_casa

33

Perda da posse de bola por jogo

In [19021]:
perda_bola_jogo_casa = round(perda_bola_casa / jogos_casa, 2)

perda_bola_jogo_casa

3.3

Duelos ganhos

In [19022]:
casa["duels"] = casa["duels"].str.replace("%", "", regex=False).astype(float)

In [19023]:
duelos_ganhos_casa = casa.duels.mean().round(2)

duelos_ganhos_casa

48.6

Cartões amarelos

In [19024]:
amarelos_primeiro_tempo_casa = casa.yellow_cards.sum()

amarelos_primeiro_tempo_casa

7

Cartões vermelhos

In [19025]:
vermelhos_primeiro_tempo_casa = casa.red_cards.sum()

vermelhos_primeiro_tempo_casa

0

Salvando a informação

In [19026]:
dados = {
    "Jogos": jogos_casa,
    "Posse de bola (média)": posse_bola_casa,
    "xG médio": xg_medio_casa,
    "xG total": xg_total_casa,
    "Grandes chances criadas": grandes_chances_criadas_casa,
    "Grandes chances por jogo": grandes_chances_jogo_casa,
    "Grandes chances convertidas": grandes_chances_convertidas_casa,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_casa,
    "Finalizações": finalizacoes_casa,
    "Finalizações por jogo": finalizacoes_jogo_casa,
    "Finalizações certas": finalizacoes_certas_casa,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_casa,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_casa,
    "Finalizações dentro da área": finalizacoes_area_casa,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_casa,
    "% de finalizações dentro da área": perc_finalizacoes_area_casa,
    "Bolas na trave": bolas_trave_casa,
    "Escanteios": escanteios_casa,
    "Escanteios por jogo": escanteios_jogo_casa,
    "Passes": passes_casa,
    "Passes certos": passes_certos_casa,
    "Assertividade nos passes": perc_passes_certos_casa,
    "Entradas no terço final": emtradas_terco_final_casa,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_casa,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_casa,
    "Perdas de bola": perda_bola_casa,
    "Perdas de bola por jogo": perda_bola_jogo_casa,
    "Duelos ganhos (média)": duelos_ganhos_casa,
    "Cartões amarelos": amarelos_primeiro_tempo_casa,
    "Cartões vermelhos": vermelhos_primeiro_tempo_casa
}

In [19027]:
ordem = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [19028]:
import unicodedata

def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados = {normalizar(k): v for k, v in dados.items()}
ordem = [normalizar(k) for k in ordem]


In [19029]:
# --- Checar chaves faltantes ---
faltando = [k for k in ordem if k not in dados]
if faltando:
    print("Debug - Chaves em dados:", list(dados.keys()))
    print("Debug - Chaves em ordem:", ordem)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Reordenar o dicionário já com valores escalares ---
dados_ordenados = {k: extrair_valor(dados[k]) for k in ordem}

# --- Criar DataFrame ---
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=["_temp"]
)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)  # garante escalar
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        # se não der para converter em número, retorna como string
        return str(valor)

# --- Nome do clube ---
coluna_clube = clubes.casa

# Renomeia a coluna temporária para o nome do clube
df_final_partidas = df_final_partidas.rename(columns={"_temp": coluna_clube})

# Aplicar formatação
df_final_partidas[coluna_clube] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[coluna_clube])
]

# Índice vira coluna "Informações"
tabela_casa = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [19030]:
tabela_casa.style.hide(axis="index")

Informações,Al-Ettifaq
Jogos,10
Posse de bola (média),51.00
xG médio,0.00
xG total,0.00
Grandes chances criadas,13
Grandes chances por jogo,1
Grandes chances convertidas,5
Aproveitamento das grandes chances,38.46
Finalizações,44
Finalizações por jogo,4.40


### Fora

Jogos

In [19031]:
# Selecionando apenas o time visitante
fora = df[df["clube"] == clubes.fora]

# Filtrando os últimos 10 jogos
fora = fora.sort_values("data", ascending=False).head(10)

fora.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,through_balls,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
405,2025-12-27,2025,Al-Nassr,Casa,65%,0.0,4,8,4,5,0,6,242,2,213,0,31,3,3,51%,0,0
403,2025-11-23,2025,Al-Nassr,Casa,60%,0.0,1,9,4,4,0,2,229,0,197,0,40,0,3,45%,1,0
401,2025-11-08,2025,Al-Nassr,Fora,58%,0.0,4,13,4,11,0,3,207,0,180,0,25,3,7,56%,1,0


In [19032]:
jogos_fora = fora.shape[0]

jogos_fora

10

Posse de bola média

In [19033]:
fora["ball_possession"] = fora["ball_possession"].str.replace("%", "", regex=False)
fora["ball_possession"] = pd.to_numeric(fora["ball_possession"])

In [19034]:
posse_bola_fora = round(fora.ball_possession.mean(), 2)

posse_bola_fora

61.8

XG Médio

In [19035]:
xg_medio_fora = fora.expected_goals.mean().round(2)

xg_medio_fora

0.0

XG Total

In [19036]:
xg_total_fora = fora.expected_goals.sum().round(2)

xg_total_fora

0.0

Grandes chances criadas

In [19037]:
grandes_chances_criadas_fora = fora.big_chances.sum()

grandes_chances_criadas_fora

24

Média de grandes chances por jogo

In [19038]:
grandes_chances_jogo_fora = round(grandes_chances_criadas_fora / jogos_fora, 2)

grandes_chances_jogo_fora

2.4

Grandes chances convertidas

In [19039]:
grandes_chances_convertidas_fora = fora.big_chances_scored.sum()

grandes_chances_convertidas_fora

8

Aproveitamento das grandes chances

In [19040]:
aproveitamento_grandes_chances_fora = round((grandes_chances_convertidas_fora / grandes_chances_criadas_fora) * 100, 2)

aproveitamento_grandes_chances_fora

33.33

Finalizações


In [19041]:
finalizacoes_fora = fora.total_shots.sum()

finalizacoes_fora

97

Média de finalizações por jogo

In [19042]:
finalizacoes_jogo_fora = round(finalizacoes_fora / jogos_fora, 2)

finalizacoes_jogo_fora

9.7

Finalizações certas

In [19043]:
finalizacoes_certas_fora = fora.shots_on_target.sum()

finalizacoes_certas_fora

40

Finalizações certas por jogo

In [19044]:
finalizacoes_certas_jogo_fora = round(finalizacoes_certas_fora / jogos_fora, 2)

finalizacoes_certas_jogo_fora

4.0

Aproveitamento das finalizações

In [19045]:
aproveitamento_finalizacoes_fora = round((finalizacoes_certas_fora / finalizacoes_fora) * 100, 2)

aproveitamento_finalizacoes_fora

41.24

Finalizações dentro da área

In [19046]:
finalizacoes_area_fora = fora.shots_inside_box.sum()

finalizacoes_area_fora

64

Finalizações dentro da área por jogo

In [19047]:
finalizacoes_area_jogo_fora = round(finalizacoes_area_fora / jogos_fora, 2)

finalizacoes_area_jogo_fora

6.4

Finalizações dentro da área em relação ao total de finalizações

In [19048]:
perc_finalizacoes_area_fora = round((finalizacoes_area_fora / finalizacoes_fora) * 100, 2)

perc_finalizacoes_area_fora

65.98

Bolas na trave

In [19049]:
bolas_trave_fora = fora.hit_woodwork.sum()

bolas_trave_fora

2

Escanteios

In [19050]:
escanteios_fora = fora.corner_kicks.sum()

escanteios_fora

39

Escanteios por jogo

In [19051]:
escanteios_jogo_fora = round(escanteios_fora / jogos_fora, 2)

escanteios_jogo_fora

3.9

Passes

In [19052]:
passes_fora = fora.passes.sum()

passes_fora

2321

Passes certos

In [19053]:
passes_certos_fora = fora.accurate_passes.sum()

passes_certos_fora

2006

Aproveitamento nos passes

In [19054]:
perc_passes_certos_fora = round((passes_certos_fora / passes_fora) * 100, 2)

perc_passes_certos_fora

86.43

Entradas no terço final

In [19055]:
emtradas_terco_final_fora = fora.final_third_entries.sum()

emtradas_terco_final_fora

316

Entradas no terço final por jogo

In [19056]:
emtradas_terco_final_jogo_fora = round(emtradas_terco_final_fora / jogos_fora, 2)

emtradas_terco_final_jogo_fora

31.6

Faltas sofridas no terço final

In [19057]:
faltas_sofridas_terco_final_fora = fora.fouled_in_final_third.sum()

faltas_sofridas_terco_final_fora

22

Perda da posse de bola

In [19058]:
perda_bola_fora = fora.dispossessed.sum()

perda_bola_fora

42

Perda da posse de bola por jogo

In [19059]:
perda_bola_jogo_fora = round(perda_bola_fora / jogos_fora, 2)

perda_bola_jogo_fora

4.2

Duelos ganhos

In [19060]:
fora["duels"] = fora["duels"].str.replace("%", "", regex=False).astype(float)

In [19061]:
duelos_ganhos_fora = fora.duels.mean().round(2)

duelos_ganhos_fora

54.9

Cartões amarelos

In [19062]:
amarelos_primeiro_tempo_fora = fora.yellow_cards.sum()

amarelos_primeiro_tempo_fora

8

Cartões vermelhos

In [19063]:
vermelhos_primeiro_tempo_fora = fora.red_cards.sum()

vermelhos_primeiro_tempo_fora

0

Salvando a informação

In [19064]:
dados_fora = {
    "Jogos": jogos_fora,
    "Posse de bola (média)": posse_bola_fora,
    "xG médio": xg_medio_fora,
    "xG total": xg_total_fora,
    "Grandes chances criadas": grandes_chances_criadas_fora,
    "Grandes chances por jogo": grandes_chances_jogo_fora,
    "Grandes chances convertidas": grandes_chances_convertidas_fora,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_fora,
    "Finalizações": finalizacoes_fora,
    "Finalizações por jogo": finalizacoes_jogo_fora,
    "Finalizações certas": finalizacoes_certas_fora,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_fora,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_fora,
    "Finalizações dentro da área": finalizacoes_area_fora,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_fora,
    "% de finalizações dentro da área": perc_finalizacoes_area_fora,
    "Bolas na trave": bolas_trave_fora,
    "Escanteios": escanteios_fora,
    "Escanteios por jogo": escanteios_jogo_fora,
    "Passes": passes_fora,
    "Passes certos": passes_certos_fora,
    "Assertividade nos passes": perc_passes_certos_fora,
    "Entradas no terço final": emtradas_terco_final_fora,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_fora,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_fora,
    "Perdas de bola": perda_bola_fora,
    "Perdas de bola por jogo": perda_bola_jogo_fora,
    "Duelos ganhos (média)": duelos_ganhos_fora,
    "Cartões amarelos": amarelos_primeiro_tempo_fora,
    "Cartões vermelhos": vermelhos_primeiro_tempo_fora
}


In [19065]:
ordem_fora = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [19066]:
def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados_fora = {normalizar(k): v for k, v in dados_fora.items()}
ordem_fora = [normalizar(k) for k in ordem_fora]

In [19067]:
# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Casa ---
faltando_casa = [k for k in ordem if k not in dados]
if faltando_casa:
    print("Debug faltando CASA:", faltando_casa)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados' (casa): {faltando_casa}")

dados_ordenados_casa = {k: extrair_valor(dados[k]) for k in ordem}
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados_casa, orient="index", columns=[clubes.casa]
)

# --- Fora ---
faltando_fora = [k for k in ordem_fora if k not in dados_fora]
if faltando_fora:
    print("Debug faltando FORA:", faltando_fora)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados_fora': {faltando_fora}")

dados_ordenados_fora = {k: extrair_valor(dados_fora[k]) for k in ordem_fora}
df_fora = pd.DataFrame.from_dict(
    dados_ordenados_fora, orient="index", columns=[clubes.fora]
)

# --- Unir casa + fora ---
df_final_partidas = df_final_partidas.join(df_fora)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        return str(valor)

# --- Aplicar formatação para CASA e FORA ---
for col in [clubes.casa, clubes.fora]:
    df_final_partidas[col] = [
        _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[col])
    ]

# --- Índice vira coluna "Informações" ---
tabela_tempo1 = df_final_partidas.reset_index().rename(columns={"index": "Informações"})


In [19068]:
tabela_tempo1.style.hide(axis="index")

Informações,Al-Ettifaq,Al-Nassr
Jogos,10,10
Posse de bola (média),51.00,61.80
xG médio,0.00,0.00
xG total,0.00,0.00
Grandes chances criadas,13,24
Grandes chances por jogo,1,2
Grandes chances convertidas,5,8
Aproveitamento das grandes chances,38.46,33.33
Finalizações,44,97
Finalizações por jogo,4.40,9.70


Salvando o arquivo no Excel

In [19069]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_tempo1.to_excel(writer, sheet_name="Tempo 1", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


## 2º Tempo

### Carregando a consulta

In [19070]:
conn = mysql.connector.connect(**config)

In [19071]:
query = f"""
SELECT
    p.data,
    t2.temporada,
    t2.clube,
    CASE
        WHEN t2.clube = p.casa THEN 'Casa'
        WHEN t2.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    t2.ball_possession,
    t2.expected_goals,
    t2.big_chances,
    t2.total_shots,
    t2.shots_on_target,
    t2.shots_inside_box,
    t2.hit_woodwork,
    t2.corner_kicks,
    t2.passes,
    t2.big_chances_scored,
    t2.accurate_passes,
    t2.final_third_entries,
    t2.fouled_in_final_third,
    t2.dispossessed,
    t2.duels,
    t2.yellow_cards,
    t2.red_cards
FROM
    2_tempo t2
    INNER JOIN partidas p ON t2.id_jogo = p.id_jogo
WHERE
    t2.clube IN ("{clubes.casa}", "{clubes.fora}");
"""

In [19072]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [19073]:
conn.close()

In [19074]:
df = df[df["temporada"] == "2025"]

In [19075]:
df.head()

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
85,2025-08-19,2025,Al-Nassr,Casa,34%,0.0,2,5,1,4,1,3,119,1,95,29,2,5,46%,2,0
86,2025-08-23,2025,Al-Nassr,Casa,61%,0.0,1,5,2,4,0,4,167,1,136,15,0,5,50%,2,0
87,2025-08-28,2025,Al-Ettifaq,Casa,47%,0.0,2,3,1,3,1,2,178,1,155,24,0,3,41%,2,0
88,2025-08-29,2025,Al-Nassr,Fora,60%,0.0,4,14,6,8,2,3,250,3,223,24,1,5,52%,1,0
357,2025-09-12,2025,Al-Ettifaq,Casa,48%,0.0,0,2,0,1,0,1,139,0,114,14,1,6,44%,4,0


### Casa

Jogos

In [19076]:
# Selecionando apenas o time mandante
casa = df[df["clube"] == clubes.casa]

# Filtrando apenas os últimos 10 jogos
casa = casa.sort_values("data", ascending=False).head(10)

casa.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
404,2025-12-25,2025,Al-Ettifaq,Fora,72%,0.0,5,10,6,6,0,2,242,2,207,44,2,4,51%,0,0
402,2025-11-22,2025,Al-Ettifaq,Casa,29%,0.0,2,3,2,2,0,0,125,1,93,16,0,1,49%,0,0
400,2025-11-08,2025,Al-Ettifaq,Fora,83%,0.0,2,9,2,6,0,7,279,1,249,42,2,3,61%,0,0


In [19077]:
jogos_casa = casa.shape[0]

jogos_casa

10

Posse de bola média

In [19078]:
casa["ball_possession"] = casa["ball_possession"].str.replace("%", "", regex=False)
casa["ball_possession"] = pd.to_numeric(casa["ball_possession"])

In [19079]:
posse_bola_casa = casa.ball_possession.mean().round(2)

posse_bola_casa

54.1

XG Médio

In [19080]:
xg_medio_casa = casa.expected_goals.mean().round(2)

xg_medio_casa

0.0

XG Total

In [19081]:
xg_total_casa = casa.expected_goals.sum().round(2)

xg_total_casa

0.0

Grandes chances criadas

In [19082]:
grandes_chances_criadas_casa = casa.big_chances.sum()

grandes_chances_criadas_casa

15

Média de grandes chances por jogo

In [19083]:
grandes_chances_jogo_casa = round(grandes_chances_criadas_casa / jogos_casa, 2)

grandes_chances_jogo_casa

1.5

Grandes chances convertidas

In [19084]:
grandes_chances_convertidas_casa = casa.big_chances_scored.sum()

grandes_chances_convertidas_casa

9

Aproveitamento das grandes chances

In [19085]:
aproveitamento_grandes_chances_casa = round((grandes_chances_convertidas_casa / grandes_chances_criadas_casa) * 100, 2)

aproveitamento_grandes_chances_casa

60.0

Finalizações

In [19086]:
finalizacoes_casa = casa.total_shots.sum()

finalizacoes_casa

52

Média de finalizações por jogo

In [19087]:
finalizacoes_jogo_casa = round(finalizacoes_casa / jogos_casa, 2)

finalizacoes_jogo_casa

5.2

Finalizações certas

In [19088]:
finalizacoes_certas_casa = casa.shots_on_target.sum()

finalizacoes_certas_casa

19

Finaizações certas por jogo

In [19089]:
finalizacoes_certas_jogo_casa = round(finalizacoes_certas_casa / jogos_casa, 2)

finalizacoes_certas_jogo_casa

1.9

Aproveitamento das finalizações

In [19090]:
aproveitamento_finalizacoes_casa = round((finalizacoes_certas_casa / finalizacoes_casa) * 100, 2)

aproveitamento_finalizacoes_casa

36.54

Finalizações dentro da área

In [19091]:
finalizacoes_area_casa = casa.shots_inside_box.sum()

finalizacoes_area_casa

32

Finalizações dentro da área por jogo

In [19092]:
finalizacoes_area_jogo_casa = round(finalizacoes_area_casa / jogos_casa, 2)

finalizacoes_area_jogo_casa

3.2

Finalizações dentro da área em relação ao total de finalizações

In [19093]:
perc_finalizacoes_area_casa = round((finalizacoes_area_casa / finalizacoes_casa) * 100, 2)

perc_finalizacoes_area_casa

61.54

Bolas na trave

In [19094]:
bolas_trave_casa = casa.hit_woodwork.sum()

bolas_trave_casa

1

Escanteios

In [19095]:
escanteios_casa = casa.corner_kicks.sum()

escanteios_casa

21

Escanteios por jogo

In [19096]:
escanteios_jogo_casa = round(escanteios_casa / jogos_casa, 2)

escanteios_jogo_casa

2.1

Passes

In [19097]:
passes_casa = casa.passes.sum()

passes_casa

2019

Passes certos

In [19098]:
passes_certos_casa = casa.accurate_passes.sum()

passes_certos_casa

1708

Aproveitamento nos passes

In [19099]:
perc_passes_certos_casa = round((passes_certos_casa / passes_casa) * 100, 2)

perc_passes_certos_casa

84.6

Entradas no terço final

In [19100]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

274

Entradas no terço final por jogo

In [19101]:
emtradas_terco_final_jogo_casa = round(emtradas_terco_final_casa / jogos_casa, 2)

emtradas_terco_final_jogo_casa

27.4

Faltas sofridas no terço final

In [19102]:
faltas_sofridas_terco_final_casa = casa.fouled_in_final_third.sum()

faltas_sofridas_terco_final_casa

10

Perda da posse de bola

In [19103]:
perda_bola_casa = casa.dispossessed.sum()

perda_bola_casa

54

Perda da posse de bola por jogo

In [19104]:
perda_bola_jogo_casa = round(perda_bola_casa / jogos_casa, 2)

perda_bola_jogo_casa

5.4

Duelos ganhos

In [19105]:
casa["duels"] = casa["duels"].str.replace("%", "", regex=False).astype(float)

In [19106]:
duelos_ganhos_casa = casa.duels.mean().round(2)

duelos_ganhos_casa

49.6

Cartões amarelos

In [19107]:
amarelos_segundo_tempo_casa = casa.yellow_cards.sum()

amarelos_segundo_tempo_casa

13

Cartões vermelhos

In [19108]:
vermelhos_segundo_tempo_casa = casa.red_cards.sum()

vermelhos_segundo_tempo_casa

1

Salvando a informação

In [19109]:
dados = {
    "Jogos": jogos_casa,
    "Posse de bola (média)": posse_bola_casa,
    "xG médio": xg_medio_casa,
    "xG total": xg_total_casa,
    "Grandes chances criadas": grandes_chances_criadas_casa,
    "Grandes chances por jogo": grandes_chances_jogo_casa,
    "Grandes chances convertidas": grandes_chances_convertidas_casa,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_casa,
    "Finalizações": finalizacoes_casa,
    "Finalizações por jogo": finalizacoes_jogo_casa,
    "Finalizações certas": finalizacoes_certas_casa,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_casa,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_casa,
    "Finalizações dentro da área": finalizacoes_area_casa,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_casa,
    "% de finalizações dentro da área": perc_finalizacoes_area_casa,
    "Bolas na trave": bolas_trave_casa,
    "Escanteios": escanteios_casa,
    "Escanteios por jogo": escanteios_jogo_casa,
    "Passes": passes_casa,
    "Passes certos": passes_certos_casa,
    "Assertividade nos passes": perc_passes_certos_casa,
    "Entradas no terço final": emtradas_terco_final_casa,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_casa,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_casa,
    "Perdas de bola": perda_bola_casa,
    "Perdas de bola por jogo": perda_bola_jogo_casa,
    "Duelos ganhos (média)": duelos_ganhos_casa,
    "Cartões amarelos": amarelos_segundo_tempo_casa,
    "Cartões vermelhos": vermelhos_segundo_tempo_casa
}

In [19110]:
ordem = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [19111]:
import unicodedata

def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados = {normalizar(k): v for k, v in dados.items()}
ordem = [normalizar(k) for k in ordem]


In [19112]:
# --- Checar chaves faltantes ---
faltando = [k for k in ordem if k not in dados]
if faltando:
    print("Debug - Chaves em dados:", list(dados.keys()))
    print("Debug - Chaves em ordem:", ordem)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Reordenar o dicionário já com valores escalares ---
dados_ordenados = {k: extrair_valor(dados[k]) for k in ordem}

# --- Criar DataFrame ---
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=["_temp"]
)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)  # garante escalar
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        # se não der para converter em número, retorna como string
        return str(valor)

# --- Nome do clube ---
coluna_clube = clubes.casa

# Renomeia a coluna temporária para o nome do clube
df_final_partidas = df_final_partidas.rename(columns={"_temp": coluna_clube})

# Aplicar formatação
df_final_partidas[coluna_clube] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[coluna_clube])
]

# Índice vira coluna "Informações"
tabela_casa = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [19113]:
tabela_casa.style.hide(axis="index")

Informações,Al-Ettifaq
Jogos,10
Posse de bola (média),54.10
xG médio,0.00
xG total,0.00
Grandes chances criadas,15
Grandes chances por jogo,1
Grandes chances convertidas,9
Aproveitamento das grandes chances,60.00
Finalizações,52
Finalizações por jogo,5.20


### Fora

Jogos

In [19114]:
# Selecionando apenas o time visitante
fora = df[df["clube"] == clubes.fora]

# Filtrando apenas os últimos 10 jogos
fora = fora.sort_values("data", ascending=False).head(10)

fora.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
405,2025-12-27,2025,Al-Nassr,Casa,64%,0.0,2,7,1,5,1,2,248,1,221,40,0,4,50%,0,0
403,2025-11-23,2025,Al-Nassr,Casa,48%,0.0,2,12,6,8,0,2,160,0,135,29,1,2,47%,2,0
401,2025-11-08,2025,Al-Nassr,Fora,59%,0.0,2,10,6,5,1,0,203,1,183,27,2,5,38%,1,0


In [19115]:
jogos_fora = fora.shape[0]

jogos_fora

10

Posse de bola média

In [19116]:
fora["ball_possession"] = fora["ball_possession"].str.replace("%", "", regex=False)
fora["ball_possession"] = pd.to_numeric(fora["ball_possession"])

In [19117]:
posse_bola_fora = fora.ball_possession.mean().round(2)

posse_bola_fora

60.7

XG Médio

In [19118]:
xg_medio_fora = fora.expected_goals.mean().round(2)

xg_medio_fora

0.0

XG Total

In [19119]:
xg_total_fora = fora.expected_goals.sum().round(2)

xg_total_fora

0.0

Grandes chances criadas

In [19120]:
grandes_chances_criadas_fora = fora.big_chances.sum()

grandes_chances_criadas_fora

24

Média de grandes chances por jogo

In [19121]:
grandes_chances_jogo_fora = round(grandes_chances_criadas_fora / jogos_fora, 2)

grandes_chances_jogo_fora

2.4

Grandes chances convertidas

In [19122]:
grandes_chances_convertidas_fora = fora.big_chances_scored.sum()

grandes_chances_convertidas_fora

8

Aproveitamento das grandes chances

In [19123]:
aproveitamento_grandes_chances_fora = round((grandes_chances_convertidas_fora / grandes_chances_criadas_fora) * 100, 2)

aproveitamento_grandes_chances_fora

33.33

Finalizações

In [19124]:
finalizacoes_fora = fora.total_shots.sum()

finalizacoes_fora

104

Média de finalizações por jogo

In [19125]:
finalizacoes_jogo_fora = round(finalizacoes_fora / jogos_fora, 2)

finalizacoes_jogo_fora

10.4

Finalizações certas

In [19126]:
finalizacoes_certas_fora = fora.shots_on_target.sum()

finalizacoes_certas_fora

40

Finalizações certas por jogo

In [19127]:
finalizacoes_certas_jogo_fora = round(finalizacoes_certas_fora / jogos_fora, 2)

finalizacoes_certas_jogo_fora

4.0

Aproveitamento das finalizações

In [19128]:
aproveitamento_finalizacoes_fora = round((finalizacoes_certas_fora / finalizacoes_fora) * 100, 2)

aproveitamento_finalizacoes_fora

38.46

Finalizações dentro da área

In [19129]:
finalizacoes_area_fora = fora.shots_inside_box.sum()

finalizacoes_area_fora

70

Finalizações dentro da área por jogo

In [19130]:
finalizacoes_area_jogo_fora = round(finalizacoes_area_fora / jogos_fora, 2)

finalizacoes_area_jogo_fora

7.0

Finalizações dentro da área em relação ao total de finalizações

In [19131]:
perc_finalizacoes_area_fora = round((finalizacoes_area_fora / finalizacoes_fora) * 100, 2)

perc_finalizacoes_area_fora

67.31

Bolas na trave

In [19132]:
bolas_trave_fora = fora.hit_woodwork.sum()

bolas_trave_fora

3

Escanteios

In [19133]:
escanteios_fora = fora.corner_kicks.sum()

escanteios_fora

39

Escanteios por jogo

In [19134]:
escanteios_jogo_fora = round(escanteios_fora / jogos_fora, 2)

escanteios_jogo_fora

3.9

Passes

In [19135]:
passes_fora = fora.passes.sum()

passes_fora

2189

Passes certos

In [19136]:
passes_certos_fora = fora.accurate_passes.sum()

passes_certos_fora

1918

Aproveitamento nos passes

In [19137]:
perc_passes_certos_fora = round((passes_certos_fora / passes_fora) * 100, 2)

perc_passes_certos_fora

87.62

Entradas no terço final

In [19138]:
emtradas_terco_final_fora = fora.final_third_entries.sum()

emtradas_terco_final_fora

316

Entradas no terço final por jogo

In [19139]:
emtradas_terco_final_jogo_fora = round(emtradas_terco_final_fora / jogos_fora, 2)

emtradas_terco_final_jogo_fora

31.6

Faltas sofridas no terço final

In [19140]:
faltas_sofridas_terco_final_fora = fora.fouled_in_final_third.sum()

faltas_sofridas_terco_final_fora

17

Perda da posse de bola

In [19141]:
perda_bola_fora = fora.dispossessed.sum()

perda_bola_fora

33

Perda da posse de bola por jogo

In [19142]:
perda_bola_jogo_fora = round(perda_bola_fora / jogos_fora, 2)

perda_bola_jogo_fora

3.3

Duelos ganhos

In [19143]:
fora["duels"] = fora["duels"].str.replace("%", "", regex=False).astype(float)

In [19144]:
duelos_ganhos_fora = fora.duels.mean().round(2)

duelos_ganhos_fora

51.7

Cartões amarelos

In [19145]:
amarelos_segundo_tempo_fora = fora.yellow_cards.sum()

amarelos_segundo_tempo_fora

7

Cartões vermelhos

In [19146]:
vermelhos_segundo_tempo_fora = fora.red_cards.sum()

vermelhos_segundo_tempo_fora

0

Salvando a informação

In [19147]:
dados_fora = {
    "Jogos": jogos_fora,
    "Posse de bola (média)": posse_bola_fora,
    "xG médio": xg_medio_fora,
    "xG total": xg_total_fora,
    "Grandes chances criadas": grandes_chances_criadas_fora,
    "Grandes chances por jogo": grandes_chances_jogo_fora,
    "Grandes chances convertidas": grandes_chances_convertidas_fora,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_fora,
    "Finalizações": finalizacoes_fora,
    "Finalizações por jogo": finalizacoes_jogo_fora,
    "Finalizações certas": finalizacoes_certas_fora,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_fora,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_fora,
    "Finalizações dentro da área": finalizacoes_area_fora,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_fora,
    "% de finalizações dentro da área": perc_finalizacoes_area_fora,
    "Bolas na trave": bolas_trave_fora,
    "Escanteios": escanteios_fora,
    "Escanteios por jogo": escanteios_jogo_fora,
    "Passes": passes_fora,
    "Passes certos": passes_certos_fora,
    "Assertividade nos passes": perc_passes_certos_fora,
    "Entradas no terço final": emtradas_terco_final_fora,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_fora,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_fora,
    "Perdas de bola": perda_bola_fora,
    "Perdas de bola por jogo": perda_bola_jogo_fora,
    "Duelos ganhos (média)": duelos_ganhos_fora,
    "Cartões amarelos": amarelos_segundo_tempo_fora,
    "Cartões vermelhos": vermelhos_segundo_tempo_fora
}


In [19148]:
ordem_fora = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [19149]:
def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados_fora = {normalizar(k): v for k, v in dados_fora.items()}
ordem_fora = [normalizar(k) for k in ordem_fora]

In [19150]:
# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Casa ---
faltando_casa = [k for k in ordem if k not in dados]
if faltando_casa:
    print("Debug faltando CASA:", faltando_casa)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados' (casa): {faltando_casa}")

dados_ordenados_casa = {k: extrair_valor(dados[k]) for k in ordem}
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados_casa, orient="index", columns=[clubes.casa]
)

# --- Fora ---
faltando_fora = [k for k in ordem_fora if k not in dados_fora]
if faltando_fora:
    print("Debug faltando FORA:", faltando_fora)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados_fora': {faltando_fora}")

dados_ordenados_fora = {k: extrair_valor(dados_fora[k]) for k in ordem_fora}
df_fora = pd.DataFrame.from_dict(
    dados_ordenados_fora, orient="index", columns=[clubes.fora]
)

# --- Unir casa + fora ---
df_final_partidas = df_final_partidas.join(df_fora)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        return str(valor)

# --- Aplicar formatação para CASA e FORA ---
for col in [clubes.casa, clubes.fora]:
    df_final_partidas[col] = [
        _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[col])
    ]

# --- Índice vira coluna "Informações" ---
tabela_tempo2 = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [19151]:
tabela_tempo2.style.hide(axis="index")

Informações,Al-Ettifaq,Al-Nassr
Jogos,10,10
Posse de bola (média),54.10,60.70
xG médio,0.00,0.00
xG total,0.00,0.00
Grandes chances criadas,15,24
Grandes chances por jogo,1,2
Grandes chances convertidas,9,8
Aproveitamento das grandes chances,60.00,33.33
Finalizações,52,104
Finalizações por jogo,5.20,10.40


Salvando o arquivo no Excel

In [19152]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_tempo2.to_excel(writer, sheet_name="Tempo 2", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


## Partida completa

### Carregando a consulta

In [19153]:
conn = mysql.connector.connect(**config)

In [19154]:
query = f"""
SELECT
    p.data,
    ep.temporada,
    ep.clube,
    CASE
        WHEN ep.clube = p.casa THEN 'Casa'
        WHEN ep.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    ep.ball_possession,
    ep.expected_goals,
    ep.big_chances,
    ep.total_shots,
    ep.shots_on_target,
    ep.shots_inside_box,
    ep.hit_woodwork,
    ep.corner_kicks,
    ep.passes,
    ep.big_chances_scored,
    ep.accurate_passes,
    ep.final_third_entries,
    ep.fouled_in_final_third,
    ep.dispossessed,
    ep.duels,
    ep.yellow_cards
FROM
    estatisticas_partidas ep
    INNER JOIN partidas p ON ep.id_jogo = p.id_jogo
WHERE
    ep.clube IN ("{clubes.casa}", "{clubes.fora}");
"""

In [19155]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [19156]:
conn.close()

In [19157]:
df = df[df["temporada"] == "2025"]

In [19158]:
df.head()

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards
85,2025-08-19,2025,Al-Nassr,Casa,36%,0.0,4,15,3,12,1,6,266,2,222,49,4,5,49%,2
86,2025-08-23,2025,Al-Nassr,Casa,62%,0.0,4,12,5,10,0,7,354,2,290,41,1,8,53%,3
87,2025-08-28,2025,Al-Ettifaq,Casa,52%,0.0,2,8,3,6,1,3,407,1,360,46,0,6,39%,3
88,2025-08-29,2025,Al-Nassr,Fora,59%,0.0,4,21,10,12,2,5,494,3,439,55,1,8,64%,1
357,2025-09-12,2025,Al-Ettifaq,Casa,42%,0.0,1,4,0,2,0,3,279,0,230,31,2,8,45%,4


### Casa

In [19159]:
# Selecionando apenas o time de casa
casa = df[df["clube"] == clubes.casa]

# Filtrando apenas os últimos 10 jogos
casa = casa.sort_values("data", ascending=False).head(10)

casa.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards
404,2025-12-25,2025,Al-Ettifaq,Fora,66%,1.69,5,15,8,8,1,2,570,2,506,79,3,6,50%,1
402,2025-11-22,2025,Al-Ettifaq,Casa,39%,0.98,4,7,5,4,0,0,345,3,277,29,2,4,49%,0
400,2025-11-08,2025,Al-Ettifaq,Fora,72%,1.33,3,12,3,8,0,9,507,1,439,70,3,5,55%,0


In [19160]:
jogos_casa = casa.shape[0]

jogos_casa

10

Posse de bola média

In [19161]:
casa["ball_possession"] = casa["ball_possession"].str.replace("%", "", regex=False)
casa["ball_possession"] = pd.to_numeric(casa["ball_possession"])

In [19162]:
posse_bola_casa = casa.ball_possession.mean().round(2)

posse_bola_casa

52.5

XG Médio

In [19163]:
xg_medio_casa = casa.expected_goals.mean().round(2)

xg_medio_casa

0.56

XG Total

In [19164]:
xg_total_casa = casa.expected_goals.sum().round(2)

xg_total_casa

5.64

Grandes chances criadas

In [19165]:
grandes_chances_criadas_casa = casa.big_chances.sum()

grandes_chances_criadas_casa

30

Média de grandes chances por jogo

In [19166]:
grandes_chances_jogo_casa = round(grandes_chances_criadas_casa / jogos_casa, 2)

grandes_chances_jogo_casa

3.0

Grandes chances convertidas

In [19167]:
grandes_chances_convertidas_casa = casa.big_chances_scored.sum()

grandes_chances_convertidas_casa

14

Aproveitamento das grandes chances

In [19168]:
aproveitamento_grandes_chances_casa = round((grandes_chances_convertidas_casa / grandes_chances_criadas_casa) * 100, 2)

aproveitamento_grandes_chances_casa

46.67

Finalizações


In [19169]:
finalizacoes_casa = casa.total_shots.sum()

finalizacoes_casa

103

Média de finalizações por jogo

In [19170]:
finalizacoes_jogo_casa = round(finalizacoes_casa / jogos_casa, 2)

finalizacoes_jogo_casa

10.3

Finalizações certas

In [19171]:
finalizacoes_certas_casa = casa.shots_on_target.sum()

finalizacoes_certas_casa

43

Finaizações certas por jogo

In [19172]:
finalizacoes_certas_jogo_casa = round(finalizacoes_certas_casa / jogos_casa, 2)

finalizacoes_certas_jogo_casa

4.3

Aproveitamento das finalizações

In [19173]:
aproveitamento_finalizacoes_casa = round((finalizacoes_certas_casa / finalizacoes_casa) * 100, 2)

aproveitamento_finalizacoes_casa

41.75

Finalizações dentro da área

In [19174]:
finalizacoes_area_casa = casa.shots_inside_box.sum()

finalizacoes_area_casa

69

Finalizações dentro da área por jogo

In [19175]:
finalizacoes_area_jogo_casa = round(finalizacoes_area_casa / jogos_casa, 2)

finalizacoes_area_jogo_casa

6.9

Finalizações dentro da área em relação ao total de finalizações

In [19176]:
perc_finalizacoes_area_casa = round((finalizacoes_area_casa / finalizacoes_casa) * 100, 2)

perc_finalizacoes_area_casa

66.99

Bolas na trave

In [19177]:
bolas_trave_casa = casa.hit_woodwork.sum()

bolas_trave_casa

5

Escanteios

In [19178]:
escanteios_casa = casa.corner_kicks.sum()

escanteios_casa

43

Escanteios por jogo

In [19179]:
escanteios_jogo_casa = round(escanteios_casa / jogos_casa, 2)

escanteios_jogo_casa

4.3

Passes

In [19180]:
passes_casa = casa.passes.sum()

passes_casa

4315

Passes certos

In [19181]:
passes_certos_casa = casa.accurate_passes.sum()

passes_certos_casa

3649

Aproveitamento nos passes

In [19182]:
perc_passes_certos_casa = round((passes_certos_casa / passes_casa) * 100, 2)

perc_passes_certos_casa

84.57

Entradas no terço final

In [19183]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

535

Entradas no terço final

In [19184]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

535

Entradas no terço final por jogo

In [19185]:
emtradas_terco_final_jogo_casa = round(emtradas_terco_final_casa / jogos_casa, 2)

emtradas_terco_final_jogo_casa

53.5

Faltas sofridas no terço final

In [19186]:
faltas_sofridas_terco_final_casa = casa.fouled_in_final_third.sum()

faltas_sofridas_terco_final_casa

22

Perda da posse de bola

In [19187]:
perda_bola_casa = casa.dispossessed.sum()

perda_bola_casa

92

Perda da posse de bola por jogo

In [19188]:
perda_bola_jogo_casa = round(perda_bola_casa / jogos_casa, 2)

perda_bola_jogo_casa

9.2

Duelos ganhos

In [19189]:
casa["duels"] = casa["duels"].str.replace("%", "", regex=False).astype(float)

In [19190]:
duelos_ganhos_casa = casa.duels.mean().round(2)

duelos_ganhos_casa

49.2

Cartões amarelos

In [19191]:
amarelos_casa = casa.yellow_cards.sum()

amarelos_casa

21

Cartões vermelhos

In [19192]:
vermelhos_casa = vermelhos_primeiro_tempo_casa + vermelhos_segundo_tempo_casa

vermelhos_casa

1

Salvando a informação

In [19193]:
dados = {
    "Jogos": jogos_casa,
    "Posse de bola (média)": posse_bola_casa,
    "xG médio": xg_medio_casa,
    "xG total": xg_total_casa,
    "Grandes chances criadas": grandes_chances_criadas_casa,
    "Grandes chances por jogo": grandes_chances_jogo_casa,
    "Grandes chances convertidas": grandes_chances_convertidas_casa,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_casa,
    "Finalizações": finalizacoes_casa,
    "Finalizações por jogo": finalizacoes_jogo_casa,
    "Finalizações certas": finalizacoes_certas_casa,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_casa,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_casa,
    "Finalizações dentro da área": finalizacoes_area_casa,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_casa,
    "% de finalizações dentro da área": perc_finalizacoes_area_casa,
    "Bolas na trave": bolas_trave_casa,
    "Escanteios": escanteios_casa,
    "Escanteios por jogo": escanteios_jogo_casa,
    "Passes": passes_casa,
    "Passes certos": passes_certos_casa,
    "Assertividade nos passes": perc_passes_certos_casa,
    "Entradas no terço final": emtradas_terco_final_casa,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_casa,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_casa,
    "Perdas de bola": perda_bola_casa,
    "Perdas de bola por jogo": perda_bola_jogo_casa,
    "Duelos ganhos (média)": duelos_ganhos_casa,
    "Cartões amarelos": amarelos_casa,
    "Cartões vermelhos": vermelhos_casa
}

In [19194]:
ordem = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [19195]:
import unicodedata

def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados = {normalizar(k): v for k, v in dados.items()}
ordem = [normalizar(k) for k in ordem]


In [19196]:
# --- Checar chaves faltantes ---
faltando = [k for k in ordem if k not in dados]
if faltando:
    print("Debug - Chaves em dados:", list(dados.keys()))
    print("Debug - Chaves em ordem:", ordem)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Reordenar o dicionário já com valores escalares ---
dados_ordenados = {k: extrair_valor(dados[k]) for k in ordem}

# --- Criar DataFrame ---
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=["_temp"]
)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)  # garante escalar
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        # se não der para converter em número, retorna como string
        return str(valor)

# --- Nome do clube ---
coluna_clube = clubes.casa

# Renomeia a coluna temporária para o nome do clube
df_final_partidas = df_final_partidas.rename(columns={"_temp": coluna_clube})

# Aplicar formatação
df_final_partidas[coluna_clube] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[coluna_clube])
]

# Índice vira coluna "Informações"
tabela_casa = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [19197]:
tabela_casa.style.hide(axis="index")

Informações,Al-Ettifaq
Jogos,10
Posse de bola (média),52.50
xG médio,0.56
xG total,5.64
Grandes chances criadas,30
Grandes chances por jogo,3
Grandes chances convertidas,14
Aproveitamento das grandes chances,46.67
Finalizações,103
Finalizações por jogo,10.30


### Fora

#### Jogos

In [19198]:
# Selecionando apenas o time visitante
fora = df[df["clube"] == clubes.fora]

# Filtrando apenas os últimos 10 jogos
fora = fora.sort_values("data", ascending=False).head(10)

fora.head(3)

C:\Users\stefa\AppData\Roaming\Python\Python312\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards
405,2025-12-27,2025,Al-Nassr,Casa,65%,2.24,6,15,5,10,1,8,490,3,434,71,3,7,51%,0
403,2025-11-23,2025,Al-Nassr,Casa,55%,1.81,3,21,10,12,0,4,389,0,332,69,1,5,46%,3
401,2025-11-08,2025,Al-Nassr,Fora,59%,3.59,6,23,10,16,1,3,410,1,363,52,5,12,49%,2


In [19199]:
jogos_fora = fora.shape[0]

jogos_fora

10

#### Posse de bola média

In [19200]:
fora["ball_possession"] = fora["ball_possession"].str.replace("%", "", regex=False)
fora["ball_possession"] = pd.to_numeric(fora["ball_possession"])

In [19201]:
posse_bola_fora = fora.ball_possession.mean().round(2)

posse_bola_fora

61.2

#### XG Médio

In [19202]:
xg_medio_fora = fora.expected_goals.mean().round(2)

xg_medio_fora

1.04

#### XG Total

In [19203]:
xg_total_fora = fora.expected_goals.sum().round(2)

xg_total_fora

10.41

#### Grandes chances criadas

In [19204]:
grandes_chances_criadas_fora = fora.big_chances.sum()

grandes_chances_criadas_fora

48

#### Média de grandes chances por jogo

In [19205]:
grandes_chances_jogo_fora = round(grandes_chances_criadas_fora / jogos_fora, 2)

grandes_chances_jogo_fora

4.8

#### Grandes chances convertidas

In [19206]:
grandes_chances_convertidas_fora = fora.big_chances_scored.sum()

grandes_chances_convertidas_fora

16

#### Aproveitamento das grandes chances

In [19207]:
aproveitamento_grandes_chances_fora = round((grandes_chances_convertidas_fora / grandes_chances_criadas_fora) * 100, 2)

aproveitamento_grandes_chances_fora

33.33

#### Finalizações

In [19208]:
finalizacoes_fora = fora.total_shots.sum()

finalizacoes_fora

201

#### Média de finalizações por jogo

In [19209]:
finalizacoes_jogo_fora = round(finalizacoes_fora / jogos_fora, 2)

finalizacoes_jogo_fora

20.1

#### Finalizações certas

In [19210]:
finalizacoes_certas_fora = fora.shots_on_target.sum()

finalizacoes_certas_fora

80

#### Finalizações certas por jogo

In [19211]:
finalizacoes_certas_jogo_fora = round(finalizacoes_certas_fora / jogos_fora, 2)

finalizacoes_certas_jogo_fora

8.0

#### Aproveitamento das finalizações

In [19212]:
aproveitamento_finalizacoes_fora = round((finalizacoes_certas_fora / finalizacoes_fora) * 100, 2)

aproveitamento_finalizacoes_fora

39.8

#### Finalizações dentro da área

In [19213]:
finalizacoes_area_fora = fora.shots_inside_box.sum()

finalizacoes_area_fora

134

#### Finalizações dentro da área por jogo

In [19214]:
finalizacoes_area_jogo_fora = round(finalizacoes_area_fora / jogos_fora, 2)

finalizacoes_area_jogo_fora

13.4

#### Finalizações dentro da área em relação ao total de finalizações

In [19215]:
perc_finalizacoes_area_fora = round((finalizacoes_area_fora / finalizacoes_fora) * 100, 2)

perc_finalizacoes_area_fora

66.67

#### Bolas na trave

In [19216]:
bolas_trave_fora = fora.hit_woodwork.sum()

bolas_trave_fora

5

#### Escanteios

In [19217]:
escanteios_fora = fora.corner_kicks.sum()

escanteios_fora

78

#### Escanteios por jogo

In [19218]:
escanteios_jogo_fora = round(escanteios_fora / jogos_fora, 2)

escanteios_jogo_fora

7.8

#### Passes

In [19219]:
passes_fora = fora.passes.sum()

passes_fora

4510

#### Passes certos

In [19220]:
passes_certos_fora = fora.accurate_passes.sum()

passes_certos_fora

3924

#### Aproveitamento nos passes

In [19221]:
perc_passes_certos_fora = round((passes_certos_fora / passes_fora) * 100, 2)

perc_passes_certos_fora

87.01

#### Entradas no terço final

In [19222]:
emtradas_terco_final_fora = fora.final_third_entries.sum()

emtradas_terco_final_fora

632

#### Entradas no terço final por jogo

In [19223]:
emtradas_terco_final_jogo_fora = round(emtradas_terco_final_fora / jogos_fora, 2)

emtradas_terco_final_jogo_fora

63.2

#### Faltas sofridas no terço final

In [19224]:
faltas_sofridas_terco_final_fora = fora.fouled_in_final_third.sum()

faltas_sofridas_terco_final_fora

39

#### Perda da posse de bola

In [19225]:
perda_bola_fora = fora.dispossessed.sum()

perda_bola_fora

75

#### Perda da posse de bola por jogo

In [19226]:
perda_bola_jogo_fora = round(perda_bola_fora / jogos_fora, 2)

perda_bola_jogo_fora

7.5

#### Duelos ganhos

In [19227]:
fora["duels"] = fora["duels"].str.replace("%", "", regex=False).astype(float)

In [19228]:
duelos_ganhos_fora = fora.duels.mean().round(2)

duelos_ganhos_fora

53.8

#### Cartões amarelos

In [19229]:
amarelos_fora = fora.yellow_cards.sum()

amarelos_fora

15

#### Cartões vermelhos

In [19230]:
vermelhos_fora = vermelhos_primeiro_tempo_fora + vermelhos_segundo_tempo_fora

vermelhos_fora

0

#### Salvando a informação

In [19231]:
dados_fora = {
    "Jogos": jogos_fora,
    "Posse de bola (média)": posse_bola_fora,
    "xG médio": xg_medio_fora,
    "xG total": xg_total_fora,
    "Grandes chances criadas": grandes_chances_criadas_fora,
    "Grandes chances por jogo": grandes_chances_jogo_fora,
    "Grandes chances convertidas": grandes_chances_convertidas_fora,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_fora,
    "Finalizações": finalizacoes_fora,
    "Finalizações por jogo": finalizacoes_jogo_fora,
    "Finalizações certas": finalizacoes_certas_fora,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_fora,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_fora,
    "Finalizações dentro da área": finalizacoes_area_fora,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_fora,
    "% de finalizações dentro da área": perc_finalizacoes_area_fora,
    "Bolas na trave": bolas_trave_fora,
    "Escanteios": escanteios_fora,
    "Escanteios por jogo": escanteios_jogo_fora,
    "Passes": passes_fora,
    "Passes certos": passes_certos_fora,
    "Assertividade nos passes": perc_passes_certos_fora,
    "Entradas no terço final": emtradas_terco_final_fora,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_fora,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_fora,
    "Perdas de bola": perda_bola_fora,
    "Perdas de bola por jogo": perda_bola_jogo_fora,
    "Duelos ganhos (média)": duelos_ganhos_fora,
    "Cartões amarelos": amarelos_fora,
    "Cartões vermelhos": vermelhos_fora
}


In [19232]:
ordem_fora = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [19233]:
def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados_fora = {normalizar(k): v for k, v in dados_fora.items()}
ordem_fora = [normalizar(k) for k in ordem_fora]

In [19234]:
# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Casa ---
faltando_casa = [k for k in ordem if k not in dados]
if faltando_casa:
    print("Debug faltando CASA:", faltando_casa)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados' (casa): {faltando_casa}")

dados_ordenados_casa = {k: extrair_valor(dados[k]) for k in ordem}
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados_casa, orient="index", columns=[clubes.casa]
)

# --- Fora ---
faltando_fora = [k for k in ordem_fora if k not in dados_fora]
if faltando_fora:
    print("Debug faltando FORA:", faltando_fora)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados_fora': {faltando_fora}")

dados_ordenados_fora = {k: extrair_valor(dados_fora[k]) for k in ordem_fora}
df_fora = pd.DataFrame.from_dict(
    dados_ordenados_fora, orient="index", columns=[clubes.fora]
)

# --- Unir casa + fora ---
df_final_partidas = df_final_partidas.join(df_fora)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        return str(valor)

# --- Aplicar formatação para CASA e FORA ---
for col in [clubes.casa, clubes.fora]:
    df_final_partidas[col] = [
        _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[col])
    ]

# --- Índice vira coluna "Informações" ---
tabela_completo = df_final_partidas.reset_index().rename(columns={"index": "Informações"})


In [19235]:
tabela_completo.style.hide(axis="index")

Informações,Al-Ettifaq,Al-Nassr
Jogos,10,10
Posse de bola (média),52.50,61.20
xG médio,0.56,1.04
xG total,5.64,10.41
Grandes chances criadas,30,48
Grandes chances por jogo,3,4
Grandes chances convertidas,14,16
Aproveitamento das grandes chances,46.67,33.33
Finalizações,103,201
Finalizações por jogo,10.30,20.10


#### Salvando o arquivo no Excel

In [19236]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_completo.to_excel(writer, sheet_name="Estatísticas Partidas", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


## Placares

#### Realizando a consulta

In [19237]:
config = {
    "host": "127.0.0.1",
    "user": "admin",
    "password": "1234",
    "database": "bet_dados"
}

In [19238]:
conn = mysql.connector.connect(**config)

In [19239]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols,
  CONCAT(COALESCE(gols_casa, 0), '-', COALESCE(gols_fora, 0)) AS placar
FROM
	partidas
WHERE
	(casa = "{clubes.casa}" OR fora = "{clubes.casa}" OR casa = "{clubes.fora}" OR fora = "{clubes.fora}");
"""

In [19240]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [19241]:
conn.close()

In [19242]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols,placar
0,2024-08-22,2024/2025,12545108,Saudita,Al-Nassr,1,1,Al-Raed,1.14,8.00,11.00,Empate,2,1-1
1,2024-08-24,2024/2025,12544932,Saudita,Al-Shabab,0,1,Al-Ettifaq,2.27,3.40,2.95,Al-Ettifaq,1,0-1
2,2024-08-27,2024/2025,12618489,Saudita,Al-Fayha,1,4,Al-Nassr,12.00,9.50,1.11,Al-Nassr,5,1-4
3,2024-08-28,2024/2025,12618497,Saudita,Al-Ettifaq,1,0,Al-Okhdood,1.38,4.33,7.50,Al-Ettifaq,1,1-0
4,2024-09-13,2024/2025,12625053,Saudita,Al-Nassr,1,1,Al-Ahli,1.70,3.90,3.90,Empate,2,1-1


#### Placares da casa desde 2020

In [19243]:
casa = df[df["casa"] == clubes.casa]

In [19244]:
casa.head(3)

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols,placar
3,2024-08-28,2024/2025,12618497,Saudita,Al-Ettifaq,1,0,Al-Okhdood,1.38,4.33,7.50,Al-Ettifaq,1,1-0
6,2024-09-20,2024/2025,12625039,Saudita,Al-Ettifaq,0,3,Al-Nassr,4.50,4.00,1.60,Al-Nassr,3,0-3
9,2024-10-03,2024/2025,12625020,Saudita,Al-Ettifaq,0,1,Al-Raed,1.57,3.80,4.75,Al-Raed,1,0-1


In [19245]:
placares_casa_historico = casa.placar.value_counts().reset_index(name="Qtd.").head(10)

placares_casa_historico["prob"] = round((placares_casa_historico["Qtd."] / placares_casa_historico["Qtd."].sum()) * 100, 2)

placares_casa_historico

,placar,Qtd.,prob
0,1-0,11,16.92
1,1-1,10,15.38
2,1-2,9,13.85
3,0-0,7,10.77
4,2-1,7,10.77
5,0-2,6,9.23
6,2-2,5,7.69
7,0-1,4,6.15
8,1-3,3,4.62
9,0-3,3,4.62


#### Placar do time da casa nos últimos 10 jogos

In [19246]:
casa_recente = casa.copy()

In [19247]:
casa_recente = casa_recente.sort_values("data", ascending=False).head(10)

casa_recente.head(3)

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols,placar
392,2025-11-22,2025,14771897,Saudita,Al-Ettifaq,3,2,Al-Fayha,2.80,3.30,2.38,Al-Ettifaq,5,3-2
388,2025-10-31,2025,14771807,Saudita,Al-Ettifaq,2,2,Al-Hazem,2.05,3.50,3.40,Empate,4,2-2
384,2025-10-18,2025,14429564,Saudita,Al-Ettifaq,0,5,Al-Hilal,5.75,4.75,1.42,Al-Hilal,5,0-5


In [19248]:
placares_casa_recente = casa_recente.placar.value_counts().reset_index(name="Qtd.").head(10)

placares_casa_recente["prob"] = round((placares_casa_recente["Qtd."] / placares_casa_recente["Qtd."].sum()) * 100, 2)

placares_casa_recente

,placar,Qtd.,prob
0,2-1,3,30.0
1,0-0,2,20.0
2,3-2,1,10.0
3,2-2,1,10.0
4,0-5,1,10.0
5,1-0,1,10.0
6,1-1,1,10.0


#### Placares do time visitante desde 2020

In [19249]:
fora = df[df["fora"] == clubes.fora]

In [19250]:
fora.head(3)

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols,placar
2,2024-08-27,2024/2025,12618489,Saudita,Al-Fayha,1,4,Al-Nassr,12.0,9.5,1.11,Al-Nassr,5,1-4
6,2024-09-20,2024/2025,12625039,Saudita,Al-Ettifaq,0,3,Al-Nassr,4.5,4.0,1.60,Al-Nassr,3,0-3
11,2024-10-18,2024/2025,12625015,Saudita,Al-Shabab,1,2,Al-Nassr,4.2,3.9,1.67,Al-Nassr,3,1-2


In [19251]:
placares_fora_historico = fora.placar.value_counts().reset_index(name="Qtd.").head(10)

placares_fora_historico["prob"] = round((placares_fora_historico["Qtd."] / placares_fora_historico["Qtd."].sum()) * 100, 2)

placares_fora_historico

,placar,Qtd.,prob
0,1-1,12,15.19
1,0-1,10,12.66
2,0-2,10,12.66
3,1-2,9,11.39
4,1-3,8,10.13
5,2-1,7,8.86
6,0-3,6,7.59
7,1-0,6,7.59
8,2-3,6,7.59
9,1-4,5,6.33


#### Placares do time visitante nos últimos 10 jogos

In [19252]:
fora_recente = fora.copy()

In [19253]:
fora_recente = fora_recente.sort_values("data", ascending=False).head(10)

fora_recente.head(3)

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols,placar
391,2025-11-08,2025,14771806,Saudita,Neom,1,3,Al-Nassr,6.25,5.0,1.33,Al-Nassr,4,1-3
386,2025-10-25,2025,14429580,Saudita,Al-Hazem,0,2,Al-Nassr,11.00,8.5,1.14,Al-Nassr,2,0-2
381,2025-09-26,2025,14429558,Saudita,Al-Ittihad,0,2,Al-Nassr,3.40,4.5,1.75,Al-Nassr,2,0-2


In [19254]:
placares_fora_recente = fora_recente.placar.value_counts().reset_index(name="Qtd.").head(10)

placares_fora_recente["prob"] = round((placares_fora_recente["Qtd."] / placares_fora_recente["Qtd."].sum()) * 100, 2)

placares_fora_recente

,placar,Qtd.,prob
0,0-2,2,20.0
1,1-3,1,10.0
2,0-4,1,10.0
3,0-5,1,10.0
4,3-2,1,10.0
5,0-9,1,10.0
6,1-4,1,10.0
7,2-3,1,10.0
8,2-1,1,10.0


#### Placar do confronto direto desde 2020

In [19255]:
placares_confronto = df[(df["casa"] == clubes.casa) & (df["fora"] == clubes.fora)]

placares_confronto

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols,placar
6,2024-09-20,2024/2025,12625039,Saudita,Al-Ettifaq,0,3,Al-Nassr,4.50,4.00,1.60,Al-Nassr,3,0-3
88,2023-08-14,2023/2024,11452941,Saudita,Al-Ettifaq,2,1,Al-Nassr,2.25,3.10,3.00,Al-Ettifaq,3,2-1
209,2023-05-27,2022/2023,10913402,Saudita,Al-Ettifaq,1,1,Al-Nassr,8.50,5.50,1.29,Empate,2,1-1
257,2022-03-11,2021/2022,10089500,Saudita,Al-Ettifaq,2,2,Al-Nassr,4.33,3.75,1.62,Empate,4,2-2
311,2021-03-05,2020/2021,9302902,Saudita,Al-Ettifaq,1,1,Al-Nassr,2.90,3.40,2.20,Empate,2,1-1


In [19256]:
placares_confronto = placares_confronto.placar.value_counts().reset_index(name="Qtd.")

placares_confronto["prob"] = round((placares_confronto["Qtd."] / placares_confronto["Qtd."].sum()) * 100, 2)

placares_confronto

,placar,Qtd.,prob
0,1-1,2,40.0
1,0-3,1,20.0
2,2-1,1,20.0
3,2-2,1,20.0


## Jogadores

### Configurando o banco de dados e carregando a consulta

In [19257]:
conn = mysql.connector.connect(**config)

In [19258]:
query = f"""
SELECT
    p.data,
    p.id_jogo,
    ej.temporada,
    CASE
        WHEN ej.team = p.casa THEN 'Casa'
        WHEN ej.team = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    ej.player_id,
    ej.name,
    CASE
        WHEN ej.position = 'G' THEN 'Goleiro'
        WHEN ej.position = 'D' THEN 'Defensor'
        WHEN ej.position = 'M' THEN 'Meio-campo'
        WHEN ej.position = 'F' THEN 'Atacante'
        ELSE 'Verificar'
    END AS position,
    ej.team,
    ej.rating,
    ej.substitute,
    ej.minutes_played,
    ej.goals,
    ej.goal_assist,
    ej.big_chance_created,
    ej.key_pass,
    ej.expected_goals
FROM
    estatisticas_jogadores ej
    INNER JOIN partidas p ON ej.id_jogo = p.id_jogo
WHERE
    ej.minutes_played > 0
    AND team IN ("{clubes.casa}", "{clubes.fora}");
"""

In [19259]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35028\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [19260]:
conn.close()

### Formação mais usada pelo mandante

#### Filtrando o time da casa e os titulares

In [19261]:
casa = df[df['team'] == clubes.casa]

casa = casa[casa['substitute'] == 0]

casa.head(3)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,substitute,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals
46,2024-10-30,12923106,2024/2025,Fora,358694,Marek Rodák,Goleiro,Al-Ettifaq,6.1,0,90,0,0,0,0,0.0
47,2024-10-30,12923106,2024/2025,Fora,963642,Radhi Al-Oteibi,Defensor,Al-Ettifaq,7.6,0,90,0,0,1,6,0.0
48,2024-10-30,12923106,2024/2025,Fora,1156413,Meshal Ali Khayrallah,Defensor,Al-Ettifaq,6.4,0,83,0,0,0,0,0.0


#### Selecionando as colunas de trabalho

In [19262]:
casa = casa[['data', 'id_jogo', 'player_id', 'name', 'position', 'team', 'minutes_played']]

#### Trabalhando a coluna de data

In [19263]:
casa['data'] = pd.to_datetime(df['data'], errors='coerce')

#### Selecionando os últimos 10 jogos

In [19264]:
ultimos_jogos = (
    casa
    .groupby(['team', 'id_jogo'], as_index=False)['data']
    .max()
)

In [19265]:
top_10_jogos = (
    ultimos_jogos
    .sort_values(['team', 'data'], ascending=[True, False])
    .groupby('team', group_keys=False)
    .head(10)[['team', 'id_jogo']]
)

In [19266]:
jogos_recentes = (
    casa.merge(top_10_jogos, on=['team', 'id_jogo'], how='inner')
)

#### Contando quem mais foi titular

In [19267]:
titulares = (
    jogos_recentes.groupby(['player_id', 'name', 'position', 'team'], as_index=False)
    .size()
    .rename(columns={'size': 'jogos_titular'})
    .sort_values(['jogos_titular', 'name'], ascending=[False, True])
    .reset_index(drop=True)
)

In [19268]:
titulares = titulares.head(11)

#### Ordenando os jogadores pela posição

In [19269]:
ordem_posicoes = ['Goleiro', 'Defensor', 'Meio-campo', 'Atacante']

titulares['position'] = pd.Categorical(titulares['position'], categories=ordem_posicoes)

titulares = titulares.sort_values('position').reset_index(drop=True)

In [19270]:
titulares_casa = titulares[['name', 'position', 'team', 'jogos_titular']]

titulares_casa

,name,position,team,jogos_titular
0,Marek Rodák,Goleiro,Al-Ettifaq,10
1,Francisco Calvo,Defensor,Al-Ettifaq,10
2,Jack Hendry,Defensor,Al-Ettifaq,10
3,Abdullah Al Khateeb,Defensor,Al-Ettifaq,7
4,Abdullah Madu,Defensor,Al-Ettifaq,5
5,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,10
6,Ondrej Duda,Meio-campo,Al-Ettifaq,9
7,Álvaro Medrán,Meio-campo,Al-Ettifaq,9
8,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,7
9,Mohau Nkota,Meio-campo,Al-Ettifaq,7


#### Montando a formação com os jogadores mais utilizados

In [19271]:
defensores = titulares[titulares['position'] == 'Defensor'].shape[0]

defensores

4

In [19272]:
meio_campo = titulares[titulares['position'] == 'Meio-campo'].shape[0]

meio_campo

6

In [19273]:
atacantes = titulares[titulares['position'] == 'Atacante'].shape[0]

atacantes

0

#### Formação mais usada pelo mandante

In [19274]:
formacao_mais_usada_casa = f'{defensores}-{meio_campo}-{atacantes}'

formacao_mais_usada_casa

'4-6-0'

### Formação mais usada pelo visitante

#### Filtrando o time da casa e os titulares

In [19275]:
fora = df[df['team'] == clubes.fora]

fora = fora[fora['substitute'] == 0]

fora.head(3)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,substitute,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals
0,2025-01-26,12624919,2024/2025,Casa,1016104,Bento,Goleiro,Al-Nassr,6.1,0,90,0,0,0,0,0.0
1,2025-01-26,12624919,2024/2025,Casa,830126,Sultan Al-Ghannam,Defensor,Al-Nassr,7.2,0,89,0,0,0,1,0.0
2,2025-01-26,12624919,2024/2025,Casa,945063,Mohamed Simakan,Defensor,Al-Nassr,7.8,0,90,1,0,0,0,0.0


#### Selecionando as colunas de trabalho

In [19276]:
fora = fora[['data', 'id_jogo', 'player_id', 'name', 'position', 'team', 'minutes_played']]

#### Trabalhando a coluna de data

In [19277]:
fora['data'] = pd.to_datetime(df['data'], errors='coerce')

#### Selecionando os últimos 10 jogos

In [19278]:
ultimos_jogos = (
    fora
    .groupby(['team', 'id_jogo'], as_index=False)['data']
    .max()
)

In [19279]:
top_10_jogos = (
    ultimos_jogos
    .sort_values(['team', 'data'], ascending=[True, False])
    .groupby('team', group_keys=False)
    .head(10)[['team', 'id_jogo']]
)

In [19280]:
jogos_recentes = (
    fora.merge(top_10_jogos, on=['team', 'id_jogo'], how='inner')
)

#### Contando quem mais foi titular

In [19281]:
titulares = (
    jogos_recentes.groupby(['player_id', 'name', 'position', 'team'], as_index=False)
    .size()
    .rename(columns={'size': 'jogos_titular'})
    .sort_values(['jogos_titular', 'name'], ascending=[False, True])
    .reset_index(drop=True)
)

In [19282]:
titulares = titulares.head(11)

In [19283]:
titulares

,player_id,name,position,team,jogos_titular
0,173883,Iñigo Martínez,Defensor,Al-Nassr,10
1,750,Cristiano Ronaldo,Atacante,Al-Nassr,9
2,851671,João Félix,Atacante,Al-Nassr,9
3,1086455,Ângelo,Meio-campo,Al-Nassr,9
4,280441,Kingsley Coman,Meio-campo,Al-Nassr,8
5,986400,Nawaf Boushal,Defensor,Al-Nassr,8
6,217704,Sadio Mané,Meio-campo,Al-Nassr,8
7,108800,Marcelo Brozović,Meio-campo,Al-Nassr,7
8,945063,Mohamed Simakan,Defensor,Al-Nassr,7
9,830126,Sultan Al-Ghannam,Defensor,Al-Nassr,7


#### Ordenando os jogadores pela posição

In [19284]:
ordem_posicoes = ['Goleiro', 'Defensor', 'Meio-campo', 'Atacante']

titulares['position'] = pd.Categorical(titulares['position'], categories=ordem_posicoes)

titulares = titulares.sort_values('position').reset_index(drop=True)

In [19285]:
titulares_fora = titulares[['name', 'position', 'team', 'jogos_titular']]

titulares_fora

,name,position,team,jogos_titular
0,Nawaf Al-Aqidi,Goleiro,Al-Nassr,6
1,Iñigo Martínez,Defensor,Al-Nassr,10
2,Nawaf Boushal,Defensor,Al-Nassr,8
3,Mohamed Simakan,Defensor,Al-Nassr,7
4,Sultan Al-Ghannam,Defensor,Al-Nassr,7
5,Ângelo,Meio-campo,Al-Nassr,9
6,Kingsley Coman,Meio-campo,Al-Nassr,8
7,Sadio Mané,Meio-campo,Al-Nassr,8
8,Marcelo Brozović,Meio-campo,Al-Nassr,7
9,Cristiano Ronaldo,Atacante,Al-Nassr,9


#### Montando a formação com os jogadores mais utilizados

In [19286]:
defensores = titulares[titulares['position'] == 'Defensor'].shape[0]

defensores

4

In [19287]:
meio_campo = titulares[titulares['position'] == 'Meio-campo'].shape[0]

meio_campo

4

In [19288]:
atacantes = titulares[titulares['position'] == 'Atacante'].shape[0]

atacantes

2

#### Formação mais usada pelo visitante

In [19289]:
formacao_mais_usada_fora = f'{defensores}-{meio_campo}-{atacantes}'

formacao_mais_usada_fora

'4-4-2'

### Mais minutos

Criando uma cópia do DataFrame original

In [19290]:
# Filtrando apenas a temporada atual
df = df[df["temporada"] == "2025"]

df.head(3)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,substitute,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals
1258,2025-09-12,14429537,2025,Casa,358694,Marek Rodák,Goleiro,Al-Ettifaq,7.9,0,90,0,0,0,0,0.0
1259,2025-09-12,14429537,2025,Casa,963642,Radhi Al-Oteibi,Defensor,Al-Ettifaq,6.9,0,64,0,0,0,0,0.0
1260,2025-09-12,14429537,2025,Casa,920498,Abdullah Al Khateeb,Defensor,Al-Ettifaq,7.7,0,90,0,0,0,0,0.0


In [19291]:
# Criando a cópia do DataFrame
mais_minutos = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19292]:
game_dates = (
    mais_minutos
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19293]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19294]:
mais_minutos = (
    mais_minutos
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores e somando os minutos

In [19295]:
mais_minutos = (
    mais_minutos
      .groupby(["player_id", "name", "position", "team"], as_index=False)["minutes_played"]
      .sum()
      .rename(columns={"minutes_played": "minutos"})
      .sort_values(["team", "minutos"], ascending=[True, False])
      .reset_index(drop=True))

In [19296]:
mais_minutos.head(3)

,player_id,name,position,team,minutos
0,358694,Marek Rodák,Goleiro,Al-Ettifaq,930
1,795696,Jack Hendry,Defensor,Al-Ettifaq,930
2,156400,Francisco Calvo,Defensor,Al-Ettifaq,929


Montando o top 5 de cada clube

In [19297]:
mais_minutos_casa = mais_minutos[mais_minutos["team"] == clubes.casa].head(5)

mais_minutos_casa

,player_id,name,position,team,minutos
0,358694,Marek Rodák,Goleiro,Al-Ettifaq,930
1,795696,Jack Hendry,Defensor,Al-Ettifaq,930
2,156400,Francisco Calvo,Defensor,Al-Ettifaq,929
3,22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,921
4,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,843


In [19298]:
mais_minutos_fora = mais_minutos[mais_minutos["team"] == clubes.fora].head(5)

mais_minutos_fora

,player_id,name,position,team,minutos
22,173883,Iñigo Martínez,Defensor,Al-Nassr,863
23,750,Cristiano Ronaldo,Atacante,Al-Nassr,809
24,851671,João Félix,Atacante,Al-Nassr,792
25,1086455,Ângelo,Meio-campo,Al-Nassr,756
26,280441,Kingsley Coman,Meio-campo,Al-Nassr,725


Unificando as informações em um único DataFrame

In [19299]:
mais_minutos = pd.concat([mais_minutos_casa, mais_minutos_fora], ignore_index=True).sort_values("minutos", ascending=False)

mais_minutos

,player_id,name,position,team,minutos
0,358694,Marek Rodák,Goleiro,Al-Ettifaq,930
1,795696,Jack Hendry,Defensor,Al-Ettifaq,930
2,156400,Francisco Calvo,Defensor,Al-Ettifaq,929
3,22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,921
5,173883,Iñigo Martínez,Defensor,Al-Nassr,863
4,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,843
6,750,Cristiano Ronaldo,Atacante,Al-Nassr,809
7,851671,João Félix,Atacante,Al-Nassr,792
8,1086455,Ângelo,Meio-campo,Al-Nassr,756
9,280441,Kingsley Coman,Meio-campo,Al-Nassr,725


### Resultado: mais minutos

Eliminando a primeira coluna

In [19300]:
mais_minutos = mais_minutos.drop(mais_minutos.columns[0], axis=1)

mais_minutos

,name,position,team,minutos
0,Marek Rodák,Goleiro,Al-Ettifaq,930
1,Jack Hendry,Defensor,Al-Ettifaq,930
2,Francisco Calvo,Defensor,Al-Ettifaq,929
3,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,921
5,Iñigo Martínez,Defensor,Al-Nassr,863
4,Álvaro Medrán,Meio-campo,Al-Ettifaq,843
6,Cristiano Ronaldo,Atacante,Al-Nassr,809
7,João Félix,Atacante,Al-Nassr,792
8,Ângelo,Meio-campo,Al-Nassr,756
9,Kingsley Coman,Meio-campo,Al-Nassr,725


Salvando no Excel

In [19301]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    mais_minutos.head(10).to_excel(writer, sheet_name="Mais Minutos", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Artilharia

In [19302]:
gols = df.copy()

Selecionando as colunas de trabalho

In [19303]:
gols = gols[["data", "id_jogo", "player_id", "name", "position", "team", "goals"]]

gols.head()

,data,id_jogo,player_id,name,position,team,goals
1258,2025-09-12,14429537,358694,Marek Rodák,Goleiro,Al-Ettifaq,0
1259,2025-09-12,14429537,963642,Radhi Al-Oteibi,Defensor,Al-Ettifaq,0
1260,2025-09-12,14429537,920498,Abdullah Al Khateeb,Defensor,Al-Ettifaq,0
1261,2025-09-12,14429537,795696,Jack Hendry,Defensor,Al-Ettifaq,0
1262,2025-09-12,14429537,156400,Francisco Calvo,Defensor,Al-Ettifaq,0


Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19304]:
game_dates = (
    gols
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19305]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19306]:
gols = (
    gols
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores e somando os minutos

In [19307]:
gols = (
    gols
      .groupby(["player_id", "name", "position", "team"], as_index=False)["goals"]
      .sum()
      .rename(columns={"goals": "gols"})
      .sort_values(["team", "gols"], ascending=[True, False])
      .reset_index(drop=True))

gols.head(3)

,player_id,name,position,team,gols
0,22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,5
1,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,4
2,318653,Moussa Dembélé,Atacante,Al-Ettifaq,3


In [19308]:
gols = gols[gols["gols"] > 0]

gols.head(3)

,player_id,name,position,team,gols
0,22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,5
1,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,4
2,318653,Moussa Dembélé,Atacante,Al-Ettifaq,3


Removendo os valores duplicados

In [19309]:
gols = gols.drop_duplicates()

### Resultado: Artilharia

Ordenando do maior para o menor valor

In [19310]:
gols = gols.sort_values("gols", ascending=False)

gols = gols.head(10)

gols

,player_id,name,position,team,gols
22,750,Cristiano Ronaldo,Atacante,Al-Nassr,11
23,851671,João Félix,Atacante,Al-Nassr,10
0,22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,5
1,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,4
2,318653,Moussa Dembélé,Atacante,Al-Ettifaq,3
24,217704,Sadio Mané,Meio-campo,Al-Nassr,2
25,280441,Kingsley Coman,Meio-campo,Al-Nassr,2
26,1086455,Ângelo,Meio-campo,Al-Nassr,2
27,1211632,Wesley,Atacante,Al-Nassr,2
3,156400,Francisco Calvo,Defensor,Al-Ettifaq,1


Eliminando a coluna "player_id"

In [19311]:
gols = gols.drop(gols.columns[0], axis=1)

gols.head(3)

,name,position,team,gols
22,Cristiano Ronaldo,Atacante,Al-Nassr,11
23,João Félix,Atacante,Al-Nassr,10
0,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,5


Salvando as informações

In [19312]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    gols.head(10).to_excel(writer, sheet_name="Artilharia", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Assistências

Criando uma cópia do DataFrame original

In [19313]:
assistencias = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19314]:
game_dates = (
    assistencias
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19315]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19316]:
assistencias = (
    assistencias
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [19317]:
assistencias = (
    assistencias
      .groupby(["player_id", "name", "position", "team"], as_index=False)["goal_assist"]
      .sum()
      .rename(columns={"goal_assist": "assistencias"})
      .sort_values(["team", "assistencias"], ascending=[True, False])
      .reset_index(drop=True))

Eliminando os jogadores que não deram nenhuma assistência

In [19318]:
assistencias = assistencias[assistencias["assistencias"] > 0]

Removendo os valores duplicados

In [19319]:
assistencias = assistencias.drop_duplicates()

### Resultado: Assistências

Ordenando do maior para o menor

In [19320]:
assistencias = assistencias.sort_values("assistencias", ascending=False)

assistencias = assistencias.head(10)

assistencias

,player_id,name,position,team,assistencias
22,280441,Kingsley Coman,Meio-campo,Al-Nassr,5
0,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,3
25,986400,Nawaf Boushal,Defensor,Al-Nassr,3
23,217704,Sadio Mané,Meio-campo,Al-Nassr,3
24,851671,João Félix,Atacante,Al-Nassr,3
1,156400,Francisco Calvo,Defensor,Al-Ettifaq,2
2,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,2
26,1086455,Ângelo,Meio-campo,Al-Nassr,2
5,1547401,Majed Dawran,Meio-campo,Al-Ettifaq,1
6,1890491,Mohau Nkota,Meio-campo,Al-Ettifaq,1


Eliminando a coluna "player_id"

In [19321]:
assistencias = assistencias.drop(assistencias.columns[0], axis=1)

assistencias.head(3)

,name,position,team,assistencias
22,Kingsley Coman,Meio-campo,Al-Nassr,5
0,Álvaro Medrán,Meio-campo,Al-Ettifaq,3
25,Nawaf Boushal,Defensor,Al-Nassr,3


Salvando as informações

In [19322]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    assistencias.head(10).to_excel(writer, sheet_name="Assistências", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Participação em gols

In [19323]:
participacao_gols = df.copy()

Criando a coluna com a participação em gols

In [19324]:
participacao_gols["participacoes"] = participacao_gols["goals"] + participacao_gols["goal_assist"]

participacao_gols.head(3)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,substitute,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals,participacoes
1258,2025-09-12,14429537,2025,Casa,358694,Marek Rodák,Goleiro,Al-Ettifaq,7.9,0,90,0,0,0,0,0.0,0
1259,2025-09-12,14429537,2025,Casa,963642,Radhi Al-Oteibi,Defensor,Al-Ettifaq,6.9,0,64,0,0,0,0,0.0,0
1260,2025-09-12,14429537,2025,Casa,920498,Abdullah Al Khateeb,Defensor,Al-Ettifaq,7.7,0,90,0,0,0,0,0.0,0


Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19325]:
game_dates = (
    participacao_gols
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19326]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19327]:
participacao_gols = (
    participacao_gols
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [19328]:
participacao_gols = (
    participacao_gols
      .groupby(["player_id", "name", "position", "team"], as_index=False)["participacoes"]
      .sum()
      .sort_values(["team", "participacoes"], ascending=[True, False])
      .reset_index(drop=True))


Removendo os jogadores sem participação em gols

In [19329]:
participacao_gols = participacao_gols[participacao_gols["participacoes"] > 0]

participacao_gols

,player_id,name,position,team,participacoes
0,22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,6
1,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,6
2,318653,Moussa Dembélé,Atacante,Al-Ettifaq,4
3,156400,Francisco Calvo,Defensor,Al-Ettifaq,3
4,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,3
5,1890491,Mohau Nkota,Meio-campo,Al-Ettifaq,2
6,1143587,Ahmed Hassan,Atacante,Al-Ettifaq,1
7,1547401,Majed Dawran,Meio-campo,Al-Ettifaq,1
22,851671,João Félix,Atacante,Al-Nassr,13
23,750,Cristiano Ronaldo,Atacante,Al-Nassr,12


### Resultado: Participação em gols

Ordenando do maior para o menor

In [19330]:
participacao_gols = participacao_gols.sort_values("participacoes", ascending=False)

participacao_gols = participacao_gols.head(10)

participacao_gols

,player_id,name,position,team,participacoes
22,851671,João Félix,Atacante,Al-Nassr,13
23,750,Cristiano Ronaldo,Atacante,Al-Nassr,12
24,280441,Kingsley Coman,Meio-campo,Al-Nassr,7
1,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,6
0,22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,6
25,217704,Sadio Mané,Meio-campo,Al-Nassr,5
2,318653,Moussa Dembélé,Atacante,Al-Ettifaq,4
26,1086455,Ângelo,Meio-campo,Al-Nassr,4
4,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,3
3,156400,Francisco Calvo,Defensor,Al-Ettifaq,3


Eliminando a coluna "player_id"

In [19331]:
participacao_gols = participacao_gols.drop(participacao_gols.columns[0], axis=1)

participacao_gols.head(3)

,name,position,team,participacoes
22,João Félix,Atacante,Al-Nassr,13
23,Cristiano Ronaldo,Atacante,Al-Nassr,12
24,Kingsley Coman,Meio-campo,Al-Nassr,7


Salvando as informações

In [19332]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    participacao_gols.head(10).to_excel(writer, sheet_name="Participação em Gols", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Rating Total

In [19333]:
rating = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19334]:
game_dates = (
    rating
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19335]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19336]:
rating = (
    rating
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [19337]:
rating = (
    rating
      .groupby(["player_id", "name", "position", "team"], as_index=False)["rating"]
      .sum()
      .rename(columns={"rating": "rating_total"})
      .sort_values(["team", "rating_total"], ascending=[True, False])
      .reset_index(drop=True))

Montando o top 5 de cada clube

In [19338]:
rating_casa = rating[rating["team"] == clubes.casa].head(5)

rating_casa

,player_id,name,position,team,rating_total
0,358694,Marek Rodák,Goleiro,Al-Ettifaq,71.0
1,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,70.0
2,22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,69.1
3,795696,Jack Hendry,Defensor,Al-Ettifaq,68.2
4,246319,Ondrej Duda,Meio-campo,Al-Ettifaq,66.0


In [19339]:
rating_fora = rating[rating["team"] == clubes.fora].head(5)

rating_fora

,player_id,name,position,team,rating_total
22,851671,João Félix,Atacante,Al-Nassr,80.8
23,830126,Sultan Al-Ghannam,Defensor,Al-Nassr,71.3
24,173883,Iñigo Martínez,Defensor,Al-Nassr,70.9
25,867179,Abdullah Al-Khaibari,Meio-campo,Al-Nassr,69.2
26,1086455,Ângelo,Meio-campo,Al-Nassr,67.1


Unificando as informações em um único DataFrame

In [19340]:
rating = pd.concat([rating_casa, rating_fora], ignore_index=True).sort_values("rating_total", ascending=False)

rating

,player_id,name,position,team,rating_total
5,851671,João Félix,Atacante,Al-Nassr,80.8
6,830126,Sultan Al-Ghannam,Defensor,Al-Nassr,71.3
0,358694,Marek Rodák,Goleiro,Al-Ettifaq,71.0
7,173883,Iñigo Martínez,Defensor,Al-Nassr,70.9
1,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,70.0
8,867179,Abdullah Al-Khaibari,Meio-campo,Al-Nassr,69.2
2,22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,69.1
3,795696,Jack Hendry,Defensor,Al-Ettifaq,68.2
9,1086455,Ângelo,Meio-campo,Al-Nassr,67.1
4,246319,Ondrej Duda,Meio-campo,Al-Ettifaq,66.0


### Resultado: Rating Total

Ordenando do maior para o menor

In [19341]:
rating = rating[rating["rating_total"] > 0]
rating["rating_total"] = rating["rating_total"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

rating.head(10).style.hide(axis="index")

player_id,name,position,team,rating_total
851671,João Félix,Atacante,Al-Nassr,"80,80"
830126,Sultan Al-Ghannam,Defensor,Al-Nassr,"71,30"
358694,Marek Rodák,Goleiro,Al-Ettifaq,"71,00"
173883,Iñigo Martínez,Defensor,Al-Nassr,"70,90"
537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,"70,00"
867179,Abdullah Al-Khaibari,Meio-campo,Al-Nassr,"69,20"
22126,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,"69,10"
795696,Jack Hendry,Defensor,Al-Ettifaq,"68,20"
1086455,Ângelo,Meio-campo,Al-Nassr,"67,10"
246319,Ondrej Duda,Meio-campo,Al-Ettifaq,"66,00"


Eliminando a coluna "player_id"

In [19342]:
rating = rating.drop(rating.columns[0], axis=1)

rating

,name,position,team,rating_total
5,João Félix,Atacante,Al-Nassr,"80,80"
6,Sultan Al-Ghannam,Defensor,Al-Nassr,"71,30"
0,Marek Rodák,Goleiro,Al-Ettifaq,"71,00"
7,Iñigo Martínez,Defensor,Al-Nassr,"70,90"
1,Álvaro Medrán,Meio-campo,Al-Ettifaq,"70,00"
8,Abdullah Al-Khaibari,Meio-campo,Al-Nassr,"69,20"
2,Georginio Wijnaldum,Meio-campo,Al-Ettifaq,"69,10"
3,Jack Hendry,Defensor,Al-Ettifaq,"68,20"
9,Ângelo,Meio-campo,Al-Nassr,"67,10"
4,Ondrej Duda,Meio-campo,Al-Ettifaq,"66,00"


Salvando as informações

In [19343]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    rating.head(10).to_excel(writer, sheet_name="Rating Total", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Rating Médio

Criando uma cópia do DataFrame original

In [19344]:
rating_medio = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19345]:
game_dates = (
    rating_medio
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19346]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19347]:
rating_medio = (
    rating_medio
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [19348]:
rating_medio = (
    rating_medio
      .groupby(["player_id", "name", "position", "team"], as_index=False)["rating"]
      .mean()
      .rename(columns={"rating": "rating_medio"})
      .sort_values(["team", "rating_medio"], ascending=[True, False])
      .reset_index(drop=True))

#### Criando a coluna de minutos jogados

Criando uma cópia do DataFrame original

In [19349]:
minutos_jogados = df.copy()

Data do jogo por "time + jogo"

In [19350]:
game_dates = (
    minutos_jogados
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19351]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19352]:
minutos_jogados = (
    minutos_jogados
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [19353]:
minutos_jogados = (
    minutos_jogados
      .groupby(["player_id", "team"], as_index=False)["minutes_played"]
      .sum()
      .rename(columns={"minutes_played": "minutos_jogados"})
      .sort_values(["team", "minutos_jogados"], ascending=[True, False])
      .reset_index(drop=True))

Transformando os minutos jogados em inteiro

In [19354]:
minutos_jogados["minutos_jogados"] = minutos_jogados["minutos_jogados"].astype(int)

minutos_jogados.head(3)

,player_id,team,minutos_jogados
0,358694,Al-Ettifaq,930
1,795696,Al-Ettifaq,930
2,156400,Al-Ettifaq,929


Unificando os dois DataFrames

In [19355]:
rating_medio = rating_medio.merge(minutos_jogados, how="inner", on="player_id")

rating_medio.head(3)

,player_id,name,position,team_x,rating_medio,team_y,minutos_jogados
0,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,7.333333,Al-Ettifaq,559
1,358694,Marek Rodák,Goleiro,Al-Ettifaq,7.100000,Al-Ettifaq,930
2,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,7.000000,Al-Ettifaq,843


Filtrando os jogadores com mais de 10% de participação nos jogos

In [19356]:
rating_medio = rating_medio[rating_medio["minutos_jogados"] >= 90]

rating_medio.head(3)

,player_id,name,position,team_x,rating_medio,team_y,minutos_jogados
0,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,7.333333,Al-Ettifaq,559
1,358694,Marek Rodák,Goleiro,Al-Ettifaq,7.100000,Al-Ettifaq,930
2,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,7.000000,Al-Ettifaq,843


Formantando o valor do rating médio com apenas duas casas decimais

In [19357]:
rating_medio["rating_medio"] = rating_medio["rating_medio"].round(2)

rating_medio.head(3)

,player_id,name,position,team_x,rating_medio,team_y,minutos_jogados
0,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,7.33,Al-Ettifaq,559
1,358694,Marek Rodák,Goleiro,Al-Ettifaq,7.10,Al-Ettifaq,930
2,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,7.00,Al-Ettifaq,843


### Resultado: Rating Médio

Ordenando do maior para o menor

In [19358]:
rating_medio = rating_medio[["name", "position", "team_x", "rating_medio"]]
rating_medio = rating_medio.sort_values("rating_medio", ascending=False)
rating_medio["rating_medio"] = rating_medio["rating_medio"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

rating_medio.head(10).style.hide(axis="index")

name,position,team_x,rating_medio
João Félix,Atacante,Al-Nassr,"8,08"
Ângelo,Meio-campo,Al-Nassr,"7,46"
Kingsley Coman,Meio-campo,Al-Nassr,"7,33"
Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,"7,33"
Cristiano Ronaldo,Atacante,Al-Nassr,"7,29"
Nawaf Boushal,Defensor,Al-Nassr,"7,26"
Nawaf Al-Aqidi,Goleiro,Al-Nassr,"7,22"
Mohamed Simakan,Defensor,Al-Nassr,"7,16"
Sultan Al-Ghannam,Defensor,Al-Nassr,"7,13"
Marek Rodák,Goleiro,Al-Ettifaq,"7,10"


In [19359]:
rating_medio = rating_medio.head(10)

Salvando as informações

In [19360]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    rating_medio.head(10).to_excel(writer, sheet_name="Rating Médio", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Grandes Chances Criadas

Criando uma cópia do DataFrame principal

In [19361]:
grandes_chances = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19362]:
game_dates = (
    grandes_chances
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19363]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19364]:
grandes_chances = (
    grandes_chances
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [19365]:
grandes_chances.head(2)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,substitute,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals
0,2025-09-12,14429537,2025,Casa,358694,Marek Rodák,Goleiro,Al-Ettifaq,7.9,0,90,0,0,0,0,0.0
1,2025-09-12,14429537,2025,Casa,963642,Radhi Al-Oteibi,Defensor,Al-Ettifaq,6.9,0,64,0,0,0,0,0.0


In [19366]:
grandes_chances = (
    grandes_chances
      .groupby(["player_id", "name", "position", "team"], as_index=False)["big_chance_created"]
      .sum()
      .rename(columns={"big_chance_created": "grandes_chances"})
      .sort_values(["team", "grandes_chances"], ascending=[True, False])
      .reset_index(drop=True))

Eliminado os jogadores que não criaram nenhuma grande chance

In [19367]:
grandes_chances = grandes_chances[grandes_chances["grandes_chances"] > 0]

grandes_chances.head(3)

,player_id,name,position,team,grandes_chances
0,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,6
1,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,5
2,156400,Francisco Calvo,Defensor,Al-Ettifaq,4


### Resultado: Grandes Chances Criadas

In [19368]:
grandes_chances = grandes_chances.sort_values("grandes_chances", ascending=False)

grandes_chances.head(3)

,player_id,name,position,team,grandes_chances
22,280441,Kingsley Coman,Meio-campo,Al-Nassr,10
0,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,6
23,851671,João Félix,Atacante,Al-Nassr,6


Eliminando a coluna "player_id"

In [19369]:
grandes_chances = grandes_chances.drop(grandes_chances.columns[0], axis=1)

grandes_chances.head(3)

,name,position,team,grandes_chances
22,Kingsley Coman,Meio-campo,Al-Nassr,10
0,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,6
23,João Félix,Atacante,Al-Nassr,6


In [19370]:
grandes_chances = grandes_chances.head(10)

Salvando as informações

In [19371]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    grandes_chances.head(10).to_excel(writer, sheet_name="Grandes Chances", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Passes chave

Criando uma cópia do DataFrame original

In [19372]:
passes_chave = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19373]:
game_dates = (
    passes_chave
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19374]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19375]:
passes_chave = (
    passes_chave
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [19376]:
passes_chave = (
    passes_chave
      .groupby(["player_id", "name", "position", "team"], as_index=False)["key_pass"]
      .sum()
      .rename(columns={"key_pass": "passes_chave"})
      .sort_values(["team", "passes_chave"], ascending=[True, False])
      .reset_index(drop=True))

Removendo os jogadores que não deram nenhum passe-chave

In [19377]:
passes_chave = passes_chave[passes_chave["passes_chave"] > 0]

passes_chave.head(3)

,player_id,name,position,team,passes_chave
0,537018,Álvaro Medrán,Meio-campo,Al-Ettifaq,13
1,246319,Ondrej Duda,Meio-campo,Al-Ettifaq,11
2,976538,Khalid Al-Ghannam,Meio-campo,Al-Ettifaq,11


### Resultado: Passes Chave

In [19378]:
passes_chave = passes_chave.sort_values("passes_chave", ascending=False)

passes_chave.head(3)

,player_id,name,position,team,passes_chave
22,851671,João Félix,Atacante,Al-Nassr,27
23,1086455,Ângelo,Meio-campo,Al-Nassr,21
24,280441,Kingsley Coman,Meio-campo,Al-Nassr,20


Eliminando a coluna "player_id"

In [19379]:
passes_chave = passes_chave.drop(passes_chave.columns[0], axis=1)

passes_chave.head(3)

,name,position,team,passes_chave
22,João Félix,Atacante,Al-Nassr,27
23,Ângelo,Meio-campo,Al-Nassr,21
24,Kingsley Coman,Meio-campo,Al-Nassr,20


In [19380]:
passes_chave = passes_chave.head(10)

Salvando as informações

In [19381]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    passes_chave.to_excel(writer, sheet_name="Passes Chave", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### XG Total

In [19382]:
xg = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19383]:
game_dates = (
    xg
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19384]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19385]:
xg = (
    xg
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [19386]:
xg = (
    xg
      .groupby(["player_id", "name", "position", "team"], as_index=False)["expected_goals"]
      .sum()
      .rename(columns={"expected_goals": "xg_total"})
      .sort_values(["team", "xg_total"], ascending=[True, False])
      .reset_index(drop=True))

Eliminando os valores igual a 0

In [19387]:
xg = xg[xg["xg_total"] > 0]

xg.head(3)

,player_id,name,position,team,xg_total


### Resultado: XG Total

In [19388]:
xg = xg.sort_values("xg_total", ascending=False)
xg["xg_total"] = xg["xg_total"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

xg.head(3).style.hide(axis="index")

player_id,name,position,team,xg_total


Eliminando a coluna "player_id"

In [19389]:
xg = xg.drop(xg.columns[0], axis=1)

xg.head(3)

,name,position,team,xg_total


In [19390]:
xg = xg.head(10)

Salvando as informações

In [19391]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    xg.to_excel(writer, sheet_name="xG Total", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### XG Médio


Criando uma cópia do DataFrame original

In [19392]:
xg_medio = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [19393]:
game_dates = (
    xg_medio
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [19394]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [19395]:
xg_medio = (
    xg_medio
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [19396]:
xg_medio = (
    xg_medio
      .groupby(["player_id", "name", "position", "team"], as_index=False)["expected_goals"]
      .mean()
      .rename(columns={"expected_goals": "xg_medio"})
      .sort_values(["team", "xg_medio"], ascending=[True, False])
      .reset_index(drop=True))


Removendo os valores igual a 0

In [19397]:
xg_medio = xg_medio[xg_medio["xg_medio"] > 0]

xg_medio.head(3)

,player_id,name,position,team,xg_medio


### Resultado: XG Médio

In [19398]:
xg_medio = xg_medio.sort_values("xg_medio", ascending=False)
xg_medio["xg_medio"] = xg_medio["xg_medio"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

xg_medio.head(3).style.hide(axis="index")

player_id,name,position,team,xg_medio


Eliminando a coluna "player_id"

In [19399]:
xg_medio = xg_medio.drop(xg_medio.columns[0], axis=1)

xg_medio.head(3)

,name,position,team,xg_medio


In [19400]:
xg_medio = xg_medio.head(10)

Salvando o arquivo

In [19401]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    xg_medio.to_excel(writer, sheet_name="xG Médio", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


# Prompt

In [19402]:
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn

# =========================================
# OPCIONAIS: deixar a string dos DFs sem cortes/quebras
# =========================================
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)  # evita wrap precoce


def to_text_no_index(obj):
    if obj is None:
        return ""
    if isinstance(obj, pd.Series):
        obj = obj.to_frame()
    if isinstance(obj, pd.DataFrame):
        # justify="left" ajuda a manter colunas previsíveis
        return obj.to_string(index=False, justify="left")
    return str(obj)


globals_dict = globals()
names = [
    "tabela", "tabela_recente", "tabela_tempo1", "tabela_tempo2",
    "tabela_completo", "df_cleans", "desempenho_casa", "desempenho_fora",
    "desempenho_confronto", "historico_recente", "mais_minutos", "gols", "assistencias",
    "participacao_gols", "rating", "rating_medio", "grandes_chances",
    "passes_chave", "xg", "xg_medio", "placares_casa_historico", "placares_casa_recente",
    "placares_fora_historico", "placares_fora_recente", "placares_confronto",
    "formacao_mais_usada_casa", "titulares_casa", "formacao_mais_usada_fora", "titulares_fora"
]

conv = {}
for n in names:
    conv[n] = to_text_no_index(globals_dict.get(n))

# =========================================
# AQUI ASSUMO QUE VOCÊ JÁ TEM O OBJETO `clubes`
# com atributos .casa, .fora, .campeonato, .data, .hora
# =========================================

texto = f"""[ PAPEL ] 

Atue como Designer e Editor de Dados Sênior. Crie uma apresentação tipo e-book para a Bet Dados no Gamma (tema: Coal).

[ CONTEXTO ]

Público leigo–intermediário; foco em clareza.

Use somente os dados fornecidos. Sem inferências, estimativas ou previsões.

Se um dado não existir, omite a seção.

Duelo no formato “Time A x Time B” (nunca “vs”).

Estética “big tech”: precisão, consistência, espaço em branco e alinhamento milimétrico.

[ ESTILO GLOBAL — CONTRASTE OBRIGATÓRIO ] Forçar cor do texto = #FFFFFF para qualquer elemento textual, sem exceções:

Aplica cor #FFFFFF com opacidade 100% para: títulos, subtítulos, parágrafos, bullets, textos de cartões, subtítulos/captions, notas, footers, botões, placeholders, links, números de ordenação (ex.: “01”, “02” no canto), legendas de gráficos, rótulos de eixos, rótulos de dados, títulos de tabela e células de tabela.

Proibido usar cinzas/“muted”, opacidade < 100% ou “secondary text”. Nada de “texto desabilitado” em cinza.

Se houver fundo claro/Imagem, coloque placa/overlay escuro (ex.: #0A0A0A 90–95%) antes de posicionar o texto.

Ícones/linhas divisórias podem usar #FFFFFF a 70–90% de opacidade; o texto nunca.

[ GRÁFICOS ]

Desligar cores padrão do tema.

Barras/fatias/linhas: usar apenas cinzas claros #F2F2F2, #EAEAEA, #E0E0E0, #D5D5D5. (Nunca cinzas escuros.)

Contorno das barras/fatias: #FFFFFF (80–100% opacidade) para separação.

Legenda, eixos e labels: #FFFFFF.

Grade discreta: #9E9E9E a 25–35% opacidade. Fundo do gráfico transparente.

Cores oficiais dos clubes apenas como acento (borda/marker), mantendo contraste AA.

[ IDENTIDADE E IMAGENS ]

Escudos oficiais (sem distorção nem marca d’água). Se não houver, usar placeholder neutro.

Proibição absoluta de imagens com marca d’água.

Paleta do deck: #001636 como acento + neutros escuros de fundo + dados em cinzas claros.

[ TAREFA — SLIDES ] (omita os que não tiverem dados)

Capa — Título “[Time A x Time B] | Sub “[Data] — [Hora]”; chips de mandante/visitante e fase; troféu oficial como hero; overlay escuro se preciso.

Publicidade Bet Dados (obrigatório) — texto, CTA “Assinar agora” → https://pay.cakto.com.br/35ww97h_563195 e placeholder de logo.

Na publicidade, frise que a Bet Dados tem dados de mais de 90 mil jogos, mais de 140 campeonatos, competições de mais de 40 países,
mais de 4.200 clubes e mais de 115 mil jogadores. Ressalte também que a Bet Dados é um grupo no Telegram, onde são enviadas análises
dos principais jogos do dia, além de análises especiais com dados de ligas, clubes e/ou jogadores.

Ficha do Jogo — competição, temporada, data/hora, estádio/mandante.

Panorama das Equipes (cards lado a lado, 2–4 linhas por equipe, texto branco).

Indicadores-Chave (tabela/tiles).

Distribuição de Gols por Minutagem.

Ofensivo vs Defensivo.

Bolas Paradas.

Disciplina.

Tendências Temporais.

Head-to-Head (se houver).

Glossário Rápido (somente termos presentes).

Notas Metodológicas (origem/janela/limitações; “sem prognósticos”).

Slide Final (QR/CTA).

[ REGRAS DE CONTEÚDO ]

Sem previsões/odds.

Em hipótese alguma utilize imagens com marca d'água.

Nomes dos clubes exatamente como informados (acentos e caixa).

Percentuais com até 2 casas decimais; unidades PT-BR.

[ ENTRADAS ] Duelo, competição, temporada, data/hora e todas as estatísticas fornecidas (médias, totais, séries, rankings etc.).

[ AUDITORIA DE CONTRASTE (passo obrigatório no final) ]

Varra o deck e liste qualquer elemento cujo texto NÃO esteja #FFFFFF (inclui textos de cartões, subtítulos/captions, numeradores “01/02/03/04”, descrições, botões, rodapés e legendas).

Corrija automaticamente para #FFFFFF e reporte “Corrigido: [elementos]”. Se algo não puder ser alterado, substitua por placa/overlay escuro até garantir contraste AA.

[ CRITÉRIOS DE QUALIDADE ]

Fidelidade 100% aos dados.

Texto 100% branco em todo o deck; gráficos em cinzas claros; contraste AA mínimo.

Coesão (grid, margens, alinhamentos).

Didática (1 frase objetiva por gráfico).

Conformidade de marca (escudos/nome de clubes corretos; zero marca d’água).

[ CHECKLIST FINAL ]

Todo texto branco #FFFFFF (inclui cartões, captions e numeradores “01/02/03/…”).

Séries de gráficos somente #F2F2F2–#D5D5D5 + contorno branco; legenda/eixos brancos.

Sem imagens com marca d’água; escudos/taça oficiais.

“x” no duelo; nomes dos clubes fiéis.

Seções sem dados omitidas; nenhum número inventado.

Exportável para PDF com contraste perfeito.

Não trazer imagens com marca d'água.

Os dados e as estatísticas são de um levantamento da Bet Dados.

NÃO precisa informar a temporada.

Reforçando: NÃO FAÇA NENHUMA PREVISÃO PARA A PARTIDA!

Nos gráficos, SEMPRE ative o rótulo de dados (mostrar valores).

Importante: a Bet Dados NÃO oferece dados em tempo real. Não coloque isso na apresentação. Não coloque NENHUM ponto negativo da Bet Dados.

Não tenho QR Code, então não peça para o usuário scannear um QR Code. Peça apenas para clicar no botão e assinar a Bet Dados. Explique que, após o pagamento,
o link de acesso ao grupo no Telegram será enviado por e-mail para o usuário.

O botão de assinatura deve estar sempre com grande destaque na página.

COLOQUE IMAGEM APENAS NA CAPA E NA CONTRACAPA, NÃO COLOQUE MAIS NENHUMA IMAGEM NA APRESENTAÇÃO!

USE SEMPRE FONTES COM COR ESCURA, COMO PRETO, AZUL ESCURO (#001636).

[ DADOS DA PARTIDA ]

Duelo: {clubes.casa} x {clubes.fora} | {clubes.campeonato}

Data: {clubes.data}

Hora: {clubes.hora}

Segue abaixo as estatísticas para a apresentação:

Temporada atual

{conv['tabela']}

Todas as informações abaixo são baseadas nos últimos 10 jogos de cada clube. Deixe isso explícito e coloque um asterisco em todos os slides informando que os dados são das últimas 10 partidas de cada clube.

Desempenho recente

{conv['tabela_recente']}

Primeiro tempo

{conv['tabela_tempo1']}

Segundo tempo

{conv['tabela_tempo2']}

Desempenho nos 90 minutos

{conv['tabela_completo']}

Clean sheets e clean goals
{conv['df_cleans']}

Desempenho em casa do mandante do duelo
{conv['desempenho_casa']}

Desempenho fora do visitante do duelo
{conv['desempenho_fora']}

Placares mais repetidos do mandante desde 2020
{conv['placares_casa_historico']}

Placares mais repetidos do mandantes nos últimos 10 jogos
{conv['placares_casa_recente']}

Placares mais repetidos do visitante desde 2020
{conv['placares_fora_historico']}

Placares mais repetidos do visitante nos últimos 10 jogos
{conv['placares_fora_recente']}

Confronto direto desde 2020
{conv['desempenho_confronto']}

Placares do confronto direto desde 2020
{conv['placares_confronto']}

Confronto direto - últimos 10 jogos
{conv['historico_recente']}

Formação mais utilizada pelo mandante
{conv['formacao_mais_usada_casa']}

Jogadores do mandante com maior frequência como titular nos últimos 10 jogos
{conv['titulares_casa']}

Formação mais utilizada pelo visitante
{conv['formacao_mais_usada_fora']}

Jogadores do visitante com maior frequência como titular nos últimos 10 jogos
{conv['titulares_fora']}

Mais minutos

{conv['mais_minutos']}

Artilharia

{conv['gols']}

Assistências

{conv['assistencias']}

Participação em gols

{conv['participacao_gols']}

Rating total

{conv['rating']}

Rating médio

{conv['rating_medio']}

Grandes chances criadas

{conv['grandes_chances']}

Passes chave

{conv['passes_chave']}

xG total

{conv['xg']}

xG médio

{conv['xg_medio']}
"""

In [19403]:
import os

# === Descobre a pasta do projeto ===
try:
    # pasta onde este script .py está salvo
    pasta_projeto = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # fallback: pasta atual (para notebook/console)
    pasta_projeto = os.getcwd()

# === Monta o caminho do arquivo dentro da pasta do projeto ===
caminho_arquivo = os.path.join(pasta_projeto, "prompt.txt")

# === Salva como .txt (UTF-8) ===
with open(caminho_arquivo, "w", encoding="utf-8", newline="\n") as f:
    f.write(texto)

print(f"Arquivo .txt salvo com sucesso em: {caminho_arquivo}")


Arquivo .txt salvo com sucesso em: c:\Pessoal\Data_science\Projetos\sofascore\analises\prompt.txt


# Finalizando

In [19404]:
print("Finalizado")

Finalizado
